In [2]:
!pip install azureopenai

In [ ]:
pip install langchain

In [ ]:
pip install openai

In [24]:
#####goal accuracy#####
import os
import logging
from openai import AzureOpenAI

# Configuration management using environment variables
api_base = os.getenv("API_BASE", "https://tcoeaiteamgpt4o.openai.azure.com/")
api_key = os.getenv("API_KEY", "DH8RAnW9UnbzuSxm8luI7Go8nX42bdLhFIEKcPQ1U0JpOU0hPeLKJQQJ99BAACYeBjFXJ3w3AAABACOGdstk")
deployment_name = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = os.getenv("API_VERSION", "2024-05-01-preview")

# Initialize the AzureOpenAI client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}",
)

# Set up logging
# logging.basicConfig(level=logging.INFO)


def gpt4o_text_analyzer(user_prompt, prompt, response, golden_response):
    """
    Analyzes text using the GPT-4 model.

    Parameters:
    user_prompt (str): The user's prompt.
    prompt (str): The prompt to guide the text analysis.
    response (str): The Copilot generated code response.
    golden response (str):  The actual code response.

    Returns:
    str: The analysis result from the model.
    """
    # Parameter validation
    if not isinstance(user_prompt, str) or not user_prompt:
        raise ValueError("User prompt must be a non-empty string.")
    if not isinstance(prompt, str) or not prompt:
        raise ValueError("Prompt must be a non-empty string.")
    if not isinstance(response, str) or not response:
        raise ValueError("Response must be a non-empty string.")
    if not isinstance(golden_response, str) or not golden_response:
        raise ValueError("Golden response must be a non-empty string.")

    # logging.info(f"Analyzing text with user prompt: {user_prompt}, prompt: {prompt}, response: {response}, "
                 # f"and golden response: {golden_response}")
    try:
        messages = [
            {"role": "system",
             "content": "You are an expert text analyzing model that can understand and analyze text end to end."},
            {"role": "user", "content": user_prompt},
            {"role": "user", "content": prompt},
            {"role": "user", "content": response},
            {"role": "user", "content": golden_response}
        ]

        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=2000
        )
        # logging.info(f"Response: {response.choices[0].message.content}")
        return str(response.choices[0].message.content)
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return None


def gpt4o_functional_accuracy(user_prompt, response, golden_response):
    functional_accuracy_prompt = f'''You are given an original code{golden_response}, a user prompt{user_prompt}, and the Copilot-generated code{response}. Your task is to assess the functional accuracy of the Copilot-generated code compared to the original code and determine whether it successfully implements the user’s requested modifications.

Instructions:
Analyze the Functional Changes:

Identify specific lines in the Copilot-generated code{response} where modifications were made in response to the user prompt{user_prompt}.
Compare these changes against the original code{golden_response} to determine if the changes are properly implemented or not.
You need to act as a verifier agent to check if the goal of the prompt is handled and implemented in the copilot generated code or not.
Provide the output strictly in JSON format with the following keys table_data,score and reasoning:\n:

"table_data": A dictionary representing a structured comparison between the original code and the Copilot-generated code. It should include the following fields:
Here are the seven keys of table_data which are Criteria,Original Code Snippet,Original Code Line Number,Copilot-Generated Code Snippet,Copilot Code Line Number,
Status,Explanation.
"Functionality": The functionality being implemented as instructed in the prompt, use these examples only as reference but you need to understand the code and come up with your own functionalities(e.g., API request handling, exception handling, pagination).
"Original Code Snippet": The relevant line(s) from the original code.
"Original Code Line Number": The exact line number(s) from the original code.
"Copilot-Generated Code Snippet": The corresponding modification from the Copilot-generated code.
"Copilot Code Line Number": The exact line number(s) in the Copilot-generated code.
"Status": "Pass" if the change correctly implements the request, "Fail" if it introduces errors or is incorrect.
"Explanation": A brief reason for the status.
"score": A float value between 0 and 1, where 1 means the Copilot-generated code fully meets the user request, and 0 means it completely fails.Significantly reduce the score if the instructions in the prompt are not handled properly.

"reasoning": A string containing a two-sentence justification supporting the score.'''

    # Step 1: Get the criminality analysis result from GPT-4
    response = gpt4o_text_analyzer(functional_accuracy_prompt, user_prompt, response, golden_response)

    if response is None:
        return {"error": "Failed to analyze text for functional accuracy."}


    return response



user_prompt = ('''Convert the code to python''')

response = '''# IDENTIFICATION DIVISION = module/file name
# PROGRAM-ID = function name
def insert_record():
    pass
 
# DATA DIVISION would be like declaring these variables:
class WorkingStorage:
    db_status = ""  # WS-DB-STATUS
    sqlcode = 0     # WS-SQLCODE
    error_msg = ""  # WS-ERROR-MSG
 
# LINKAGE SECTION is like function parameters
class PersonRecord:
    def __init__(self, person_id: int, person_name: str, person_age: int):
        self.person_id = person_id    # PERSON-ID
        self.person_name = person_name # PERSON-NAME
        self.person_age = person_age   # PERSON-AGE
        self.success = False          # LS-RESULT'''

golden_response = '''IDENTIFICATION DIVISION.
PROGRAM-ID. INSERT-RECORD.
 
ENVIRONMENT DIVISION.
 
DATA DIVISION.
WORKING-STORAGE SECTION.
01  WS-STATUS-FLAGS.
    05 WS-DB-STATUS     PIC X(2).
       88 WS-SUCCESS    VALUE "00".
    05 WS-SQLCODE       PIC S9(9) COMP.
    05 WS-ERROR-MSG     PIC X(50).
 
LINKAGE SECTION.
01  LS-PARAMETERS.
    05 LS-PERSON-RECORD.
       10 PERSON-ID     PIC 9(6).
       10 PERSON-NAME   PIC X(50).
       10 PERSON-AGE    PIC 9(3).
    05 LS-RESULT        PIC X.
       88 SUCCESS       VALUE 'T'.
       88 FAILED        VALUE 'F'.
 
PROCEDURE DIVISION USING LS-PARAMETERS.
    PERFORM INSERT-AND-VALIDATE
    GOBACK
    .
 
INSERT-AND-VALIDATE.
    EXEC SQL
        INSERT INTO persons (id, name, age)
        VALUES (:PERSON-ID, :PERSON-NAME, :PERSON-AGE)
    END-EXEC
 
    IF SQLCODE = 0
        EXEC SQL COMMIT END-EXEC
        SET SUCCESS TO TRUE
    ELSE
        EXEC SQL ROLLBACK END-EXEC
        SET FAILED TO TRUE
        STRING "DB Error: " SQLCODE
            DELIMITED BY SIZE
            INTO WS-ERROR-MSG
        DISPLAY WS-ERROR-MSG
    END-IF
    .'''

response3 = '''# IDENTIFICATION DIVISION = module/file name
# PROGRAM-ID = function name
def insert_record():
    pass
 
# DATA DIVISION would be like declaring these variables:
class WorkingStorage:
    db_status = ""  # WS-DB-STATUS
    sqlcode = 0     # WS-SQLCODE
    error_msg = ""  # WS-ERROR-MSG
 
# LINKAGE SECTION is like function parameters
class PersonRecord:
    def __init__(self, person_id: int, person_name: str, person_age: int):
        self.person_id = person_id    # PERSON-ID
        self.person_name = person_name # PERSON-NAME
        self.person_age = person_age   # PERSON-AGE
        self.success = False          # LS-RESULT'''

response2 = '''def calculate_area_of_rectangle(length, width):
    area = length * width  
    return area  

def calculate_area_of_circle(radius):
    import math
    area = math.pi * (radius ** 2)
    return area

length_of_rectangle = 10
width_of_rectangle = 5
area_of_rectangle = calculate_area_of_rectangle(length_of_rectangle, width_of_rectangle)
print(f"Area of rectangle: {area_of_rectangle}")

radius_of_circle = 7
area_of_circle = calculate_area_of_circle(radius_of_circle)
print(f"Area of circle: {area_of_circle}")
''' 


golden_response1 = '''def calculate_area_of_rectangle(length, width):
    area = length * width    return area
def calculate_area_of_circle(radius):
    import math
    area = math.pi * (radius ** 2)
    return area

length_of_rectangle = 10width_of_rectangle = 5area_of_rectangle = calculate_area_of_rectangle(length_of_rectangle, width_of_rectangle)print(f"Area of rectangle: {area_of_rectangle}")

radius_of_circle = 7
area_of_circle = calculate_area_of_circle(radius_of_circle)
print(f"Area of circle: {area_of_circle}")'''

   

# Evaluate and print results for all four metrics
result = gpt4o_functional_accuracy(user_prompt, response, golden_response)
print("Functional Accuracy Result:", result)

Functional Accuracy Result: ```json
{
  "table_data": [
    {
      "Criteria": "Declaring Working Storage Variables",
      "Original Code Snippet": "01  WS-STATUS-FLAGS.\n    05 WS-DB-STATUS     PIC X(2).\n       88 WS-SUCCESS    VALUE \"00\".\n    05 WS-SQLCODE       PIC S9(9) COMP.\n    05 WS-ERROR-MSG     PIC X(50).",
      "Original Code Line Number": "5-9",
      "Copilot-Generated Code Snippet": "class WorkingStorage:\n    db_status = \"\"  # WS-DB-STATUS\n    sqlcode = 0     # WS-SQLCODE\n    error_msg = \"\"  # WS-ERROR-MSG",
      "Copilot Code Line Number": "4-7",
      "Status": "Pass",
      "Explanation": "The variables from the COBOL working storage section are successfully translated and included as class attributes in Python."
    },
    {
      "Criteria": "Declaring Person Record Structure",
      "Original Code Snippet": "01  LS-PARAMETERS.\n    05 LS-PERSON-RECORD.\n       10 PERSON-ID     PIC 9(6).\n       10 PERSON-NAME   PIC X(50).\n       10 PERSON-AGE    PIC 

In [31]:
#####goal accuracy#####
import os
import logging
from openai import AzureOpenAI

# Configuration management using environment variables
api_base = os.getenv("API_BASE", "https://tcoeaiteamgpt4o.openai.azure.com/")
api_key = os.getenv("API_KEY", "DH8RAnW9UnbzuSxm8luI7Go8nX42bdLhFIEKcPQ1U0JpOU0hPeLKJQQJ99BAACYeBjFXJ3w3AAABACOGdstk")
deployment_name = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = os.getenv("API_VERSION", "2024-05-01-preview")

# Initialize the AzureOpenAI client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}",
)

# Set up logging
# logging.basicConfig(level=logging.INFO)


def gpt4o_text_analyzer(user_prompt, prompt, response):
    """
    Analyzes text using the GPT-4 model.

    Parameters:
    user_prompt (str): The user's prompt.
    prompt (str): The prompt to guide the text analysis.
    response (str): The Copilot generated code response.
    golden response (str):  The actual code response.

    Returns:
    str: The analysis result from the model.
    """
    # Parameter validation
    if not isinstance(user_prompt, str) or not user_prompt:
        raise ValueError("User prompt must be a non-empty string.")
    if not isinstance(prompt, str) or not prompt:
        raise ValueError("Prompt must be a non-empty string.")
    if not isinstance(response, str) or not response:
        raise ValueError("Response must be a non-empty string.")
    if not isinstance(golden_response, str) or not golden_response:
        raise ValueError("Golden response must be a non-empty string.")

    # logging.info(f"Analyzing text with user prompt: {user_prompt}, prompt: {prompt}, response: {response}, "
                 # f"and golden response: {golden_response}")
    try:
        messages = [
            {"role": "system",
             "content": "You are an expert text analyzing model that can understand and analyze text end to end."},
            {"role": "user", "content": user_prompt},
            {"role": "user", "content": prompt},
            {"role": "user", "content": response},
            {"role": "user", "content": golden_response}
        ]

        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=2000
        )
        # logging.info(f"Response: {response.choices[0].message.content}")
        return str(response.choices[0].message.content)
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return None


def gpt4o_functional_accuracy(user_prompt, response):
    functional_accuracy_prompt = f'''You are given an  a user prompt{user_prompt}, and the Copilot-generated code{response}. Your task is to assess the functional accuracy of the Copilot-generated code compared to the original code and determine whether it successfully implements the user’s requested modifications.

Instructions:
Analyze the Functional Changes:

Identify specific lines in the Copilot-generated code{response} where modifications were made in response to the user prompt{user_prompt}.

You need to act as a verifier agent to check if the goal of the prompt is handled and implemented in the copilot generated code or not.
Provide the output strictly in JSON format with the following keys table_data,score and reasoning:\n:

"table_data": A dictionary representing a structured comparison between the original code and the Copilot-generated code. It should include the following fields:
Here are the seven keys of table_data which are Functioanlity,Copilot-Generated Code Snippet,Copilot Code Line Number,
Status,Explanation.
"Functionality": The functionality being implemented as instructed in the prompt, use these examples only as reference but you need to understand the code and come up with your own functionalities(e.g., API request handling, exception handling, pagination).
"Copilot-Generated Code Snippet": The corresponding instructions implemented in the Copilot-generated code.
"Copilot Code Line Number": The exact line number(s) in the Copilot-generated code.
"Status": "Pass" if the change correctly implements the request, "Fail" if it introduces errors or is incorrect.
"Explanation": A brief reason for the status.
"score": A float value between 0 and 1, where 1 means the Copilot-generated code fully meets the user request, and 0 means it completely fails.Significantly reduce the score if the instructions in the prompt are not handled properly.

"reasoning": A string containing a two-sentence justification supporting the score.'''

    # Step 1: Get the criminality analysis result from GPT-4
    response = gpt4o_text_analyzer(functional_accuracy_prompt, user_prompt, response)

    if response is None:
        return {"error": "Failed to analyze text for functional accuracy."}


    return response



user_prompt = ('''Generate a python program atht generates palindrom of length 5''')

response = '''import random

def generate_palindrome():
    letters = "abcdefghijklmnopqrstuvwxyz"
    first_two = "".join(random.choices(letters, k=2))
    middle = random.choice(letters)
    palindrome = first_two + middle + first_two[::-1]
    return palindrome

print("Generated Palindrome:", generate_palindrome())
'''

response1 = '''# IDENTIFICATION DIVISION = module/file name
# PROGRAM-ID = function name
def insert_record():
    pass
 
# DATA DIVISION would be like declaring these variables:
class WorkingStorage:
    db_status = ""  # WS-DB-STATUS
    sqlcode = 0     # WS-SQLCODE
    error_msg = ""  # WS-ERROR-MSG
 
# LINKAGE SECTION is like function parameters
class PersonRecord:
    def __init__(self, person_id: int, person_name: str, person_age: int):
        self.person_id = person_id    # PERSON-ID
        self.person_name = person_name # PERSON-NAME
        self.person_age = person_age   # PERSON-AGE
        self.success = False          # LS-RESULT'''

golden_response = '''IDENTIFICATION DIVISION.
PROGRAM-ID. INSERT-RECORD.
 
ENVIRONMENT DIVISION.
 
DATA DIVISION.
WORKING-STORAGE SECTION.
01  WS-STATUS-FLAGS.
    05 WS-DB-STATUS     PIC X(2).
       88 WS-SUCCESS    VALUE "00".
    05 WS-SQLCODE       PIC S9(9) COMP.
    05 WS-ERROR-MSG     PIC X(50).
 
LINKAGE SECTION.
01  LS-PARAMETERS.
    05 LS-PERSON-RECORD.
       10 PERSON-ID     PIC 9(6).
       10 PERSON-NAME   PIC X(50).
       10 PERSON-AGE    PIC 9(3).
    05 LS-RESULT        PIC X.
       88 SUCCESS       VALUE 'T'.
       88 FAILED        VALUE 'F'.
 
PROCEDURE DIVISION USING LS-PARAMETERS.
    PERFORM INSERT-AND-VALIDATE
    GOBACK
    .
 
INSERT-AND-VALIDATE.
    EXEC SQL
        INSERT INTO persons (id, name, age)
        VALUES (:PERSON-ID, :PERSON-NAME, :PERSON-AGE)
    END-EXEC
 
    IF SQLCODE = 0
        EXEC SQL COMMIT END-EXEC
        SET SUCCESS TO TRUE
    ELSE
        EXEC SQL ROLLBACK END-EXEC
        SET FAILED TO TRUE
        STRING "DB Error: " SQLCODE
            DELIMITED BY SIZE
            INTO WS-ERROR-MSG
        DISPLAY WS-ERROR-MSG
    END-IF
    .'''

response3 = '''# IDENTIFICATION DIVISION = module/file name
# PROGRAM-ID = function name
def insert_record():
    pass
 
# DATA DIVISION would be like declaring these variables:
class WorkingStorage:
    db_status = ""  # WS-DB-STATUS
    sqlcode = 0     # WS-SQLCODE
    error_msg = ""  # WS-ERROR-MSG
 
# LINKAGE SECTION is like function parameters
class PersonRecord:
    def __init__(self, person_id: int, person_name: str, person_age: int):
        self.person_id = person_id    # PERSON-ID
        self.person_name = person_name # PERSON-NAME
        self.person_age = person_age   # PERSON-AGE
        self.success = False          # LS-RESULT'''

response2 = '''def calculate_area_of_rectangle(length, width):
    area = length * width  
    return area  

def calculate_area_of_circle(radius):
    import math
    area = math.pi * (radius ** 2)
    return area

length_of_rectangle = 10
width_of_rectangle = 5
area_of_rectangle = calculate_area_of_rectangle(length_of_rectangle, width_of_rectangle)
print(f"Area of rectangle: {area_of_rectangle}")

radius_of_circle = 7
area_of_circle = calculate_area_of_circle(radius_of_circle)
print(f"Area of circle: {area_of_circle}")
''' 


golden_response1 = '''def calculate_area_of_rectangle(length, width):
    area = length * width    return area
def calculate_area_of_circle(radius):
    import math
    area = math.pi * (radius ** 2)
    return area

length_of_rectangle = 10width_of_rectangle = 5area_of_rectangle = calculate_area_of_rectangle(length_of_rectangle, width_of_rectangle)print(f"Area of rectangle: {area_of_rectangle}")

radius_of_circle = 7
area_of_circle = calculate_area_of_circle(radius_of_circle)
print(f"Area of circle: {area_of_circle}")'''

   

# Evaluate and print results for all four metrics
result = gpt4o_functional_accuracy(user_prompt, response)
print("Functional Accuracy Result:", result)

Functional Accuracy Result: ```json
{
  "table_data": [
    {
      "Functionality": "Generate a palindrome of length 5",
      "Copilot-Generated Code Snippet": "letters = \"abcdefghijklmnopqrstuvwxyz\"\nfirst_two = \"\".join(random.choices(letters, k=2))\nmiddle = random.choice(letters)\npalindrome = first_two + middle + first_two[::-1]",
      "Copilot Code Line Number": "3-6",
      "Status": "Pass",
      "Explanation": "The code correctly implements the functionality of generating a palindrome of length 5 by combining 'first_two', 'middle', and the reverse of 'first_two'."
    },
    {
      "Functionality": "Use the middle character as the center of the palindrome",
      "Copilot-Generated Code Snippet": "middle = random.choice(letters)",
      "Copilot Code Line Number": "5",
      "Status": "Pass",
      "Explanation": "The middle character is correctly extracted as a single random letter, ensuring it functions as the center of the palindrome."
    },
    {
      "Functionali

In [32]:
import pandas as pd

# JSON Data
data4 = {
  "table_data": [
    {
      "Functionality": "Generate a palindrome of length 5",
      "Copilot-Generated Code Snippet": "letters = \"abcdefghijklmnopqrstuvwxyz\"\nfirst_two = \"\".join(random.choices(letters, k=2))\nmiddle = random.choice(letters)\npalindrome = first_two + middle + first_two[::-1]",
      "Copilot Code Line Number": "3-6",
      "Status": "Pass",
      "Explanation": "The code correctly implements the functionality of generating a palindrome of length 5 by combining 'first_two', 'middle', and the reverse of 'first_two'."
    },
    {
      "Functionality": "Use the middle character as the center of the palindrome",
      "Copilot-Generated Code Snippet": "middle = random.choice(letters)",
      "Copilot Code Line Number": "5",
      "Status": "Pass",
      "Explanation": "The middle character is correctly extracted as a single random letter, ensuring it functions as the center of the palindrome."
    },
    {
      "Functionality": "Construct a string as 'first_two' + 'middle' + reverse('first_two')",
      "Copilot-Generated Code Snippet": "palindrome = first_two + middle + first_two[::-1]",
      "Copilot Code Line Number": "6",
      "Status": "Pass",
      "Explanation": "The final palindrome string is constructed correctly by combining both reversed and non-reversed portions around the middle character."
    },
    {
      "Functionality": "Output the generated palindrome",
      "Copilot-Generated Code Snippet": "print(\"Generated Palindrome:\", generate_palindrome())",
      "Copilot Code Line Number": "8",
      "Status": "Pass",
      "Explanation": "The generated palindrome is output to the console, fulfilling the goal of visibly displaying the result."
    }
  ],
  "score": 1.0,
  "reasoning": "The Copilot-generated code meets all the requirements specified in the prompt: it successfully generates a palindrome of length 5 by properly constructing the desired string structure and outputs the result. The implementation is accurate without any errors or missing functionality."
}
data3 = {
  "table_data": [
    {
      "Criteria": "Declaring Working Storage Variables",
      "Original Code Snippet": "01  WS-STATUS-FLAGS.\n    05 WS-DB-STATUS     PIC X(2).\n       88 WS-SUCCESS    VALUE \"00\".\n    05 WS-SQLCODE       PIC S9(9) COMP.\n    05 WS-ERROR-MSG     PIC X(50).",
      "Original Code Line Number": "5-9",
      "Copilot-Generated Code Snippet": "class WorkingStorage:\n    db_status = \"\"  # WS-DB-STATUS\n    sqlcode = 0     # WS-SQLCODE\n    error_msg = \"\"  # WS-ERROR-MSG",
      "Copilot Code Line Number": "4-7",
      "Status": "Pass",
      "Explanation": "The variables from the COBOL working storage section are successfully translated and included as class attributes in Python."
    },
    {
      "Criteria": "Declaring Person Record Structure",
      "Original Code Snippet": "01  LS-PARAMETERS.\n    05 LS-PERSON-RECORD.\n       10 PERSON-ID     PIC 9(6).\n       10 PERSON-NAME   PIC X(50).\n       10 PERSON-AGE    PIC 9(3).",
      "Original Code Line Number": "11-15",
      "Copilot-Generated Code Snippet": "class PersonRecord:\n    def __init__(self, person_id: int, person_name: str, person_age: int):\n        self.person_id = person_id    # PERSON-ID\n        self.person_name = person_name # PERSON-NAME\n        self.person_age = person_age   # PERSON-AGE",
      "Copilot Code Line Number": "9-14",
      "Status": "Pass",
      "Explanation": "The COBOL structure is mapped to a Python class with equivalent attributes."
    },
    {
      "Criteria": "Handling LS-RESULT Flag",
      "Original Code Snippet": "05 LS-RESULT        PIC X.\n       88 SUCCESS       VALUE 'T'.\n       88 FAILED        VALUE 'F'.",
      "Original Code Line Number": "16-18",
      "Copilot-Generated Code Snippet": "self.success = False          # LS-RESULT",
      "Copilot Code Line Number": "14",
      "Status": "Fail",
      "Explanation": "The SUCCESS and FAILED flags are not implemented as distinct boolean representations in the Python code."
    },
    {
      "Criteria": "SQL Insert Statement",
      "Original Code Snippet": "EXEC SQL\n    INSERT INTO persons (id, name, age)\n    VALUES (:PERSON-ID, :PERSON-NAME, :PERSON-AGE)\nEND-EXEC",
      "Original Code Line Number": "21-24",
      "Copilot-Generated Code Snippet": "def insert_record():\n    pass",
      "Copilot Code Line Number": "2-3",
      "Status": "Fail",
      "Explanation": "The SQL Insert functionality is missing in the Copilot-generated code despite being part of the COBOL logic."
    },
    {
      "Criteria": "Handling SQL Commit and Rollback",
      "Original Code Snippet": "IF SQLCODE = 0\n    EXEC SQL COMMIT END-EXEC\nELSE\n    EXEC SQL ROLLBACK END-EXEC",
      "Original Code Line Number": "26-29",
      "Copilot-Generated Code Snippet": "N/A",
      "Copilot Code Line Number": "N/A",
      "Status": "Fail",
      "Explanation": "The commit and rollback operations are not implemented in the Python code."
    },
    {
      "Criteria": "Error Handling and Logging",
      "Original Code Snippet": "SET FAILED TO TRUE\nSTRING \"DB Error: \" SQLCODE\n    DELIMITED BY SIZE\n    INTO WS-ERROR-MSG\nDISPLAY WS-ERROR-MSG",
      "Original Code Line Number": "30-34",
      "Copilot-Generated Code Snippet": "N/A",
      "Copilot Code Line Number": "N/A",
      "Status": "Fail",
      "Explanation": "The error handling logic and error message construction are missing in the Python code."
    }
  ],
  "score": 0.4,
  "reasoning": "While the Copilot-generated code successfully translated data structure declarations (working storage and parameters) into Python, it failed to implement core functionalities like SQL execution, transaction management, and error handling. This incomplete translation significantly reduces the functional accuracy."
}
data2 = {
  "table_data": [
    {
        "Criteria": "Syntax Error Handling",
        "Functionality": "Correct and readable definitions of functions.",
        "Original Code Snippet": "area = length * width    return area",
        "Original Code Line Number": 2,
        "Copilot-Generated Code Snippet": "area = length * width\n    return area",
        "Copilot Code Line Number": 2-3,
        "Status": "Pass",
        "Explanation": "The Copilot-generated code fixes the syntax error by splitting the two statements into separate lines, following Python's indentation rules and coding standards."
    },
    {
        "Criteria": "Whitespace and Readability",
        "Functionality": "Improve overall readability with corrected whitespaces.",
        "Original Code Snippet": "length_of_rectangle = 10width_of_rectangle = 5area_of_rectangle = calculate_area_of_rectangle(length_of_rectangle, width_of_rectangle)",
        "Original Code Line Number": 9-10,
        "Copilot-Generated Code Snippet": "length_of_rectangle = 10\nwidth_of_rectangle = 5\narea_of_rectangle = calculate_area_of_rectangle(length_of_rectangle, width_of_rectangle)",
        "Copilot Code Line Number": 7-9,
        "Status": "Pass",
        "Explanation": "The Copilot-generated code separates distinct statements into individual lines, following Python best practices and enhancing readability."
    },
    {
        "Criteria": "Code Style Consistency",
        "Functionality": "Adherence to Python standards for consistent formatting and imports.",
        "Original Code Snippet": "import math",
        "Original Code Line Number": 4,
        "Copilot-Generated Code Snippet": "import math",
        "Copilot Code Line Number": 4,
        "Status": "Pass",
        "Explanation": "Both versions use 'import math' consistently at the point where it's needed, adhering to best practices for scoped imports."
    },
    {
        "Criteria": "Functionality Preservation",
        "Functionality": "Retain functional correctness in area calculations.",
        "Original Code Snippet": "area = math.pi * (radius ** 2)",
        "Original Code Line Number": 5,
        "Copilot-Generated Code Snippet": "area = math.pi * (radius ** 2)",
        "Copilot Code Line Number": 5,
        "Status": "Pass",
        "Explanation": "The calculation logic for the area of a circle remains unchanged in Copilot's code, preserving functionality accurately."
    },
    {
        "Criteria": "Output Clarity",
        "Functionality": "Enhance clarity of output statements.",
        "Original Code Snippet": "print(f\"Area of rectangle: {area_of_rectangle}\")",
        "Original Code Line Number": 11,
        "Copilot-Generated Code Snippet": "print(f\"Area of rectangle: {area_of_rectangle}\")",
        "Copilot Code Line Number": 10,
        "Status": "Pass",
        "Explanation": "The output statement for the rectangle area is already clear and does not require modification."
    },
    {
        "Criteria": "Output Clarity",
        "Functionality": "Enhance clarity of output statements.",
        "Original Code Snippet": "print(f\"Area of circle: {area_of_circle}\")",
        "Original Code Line Number": 12,
        "Copilot-Generated Code Snippet": "print(f\"Area of circle: {area_of_circle}\")",
        "Copilot Code Line Number": 11,
        "Status": "Pass",
        "Explanation": "The output statement for the circle area is already clear and does not require modification."
    }
  ],
  "score": 1.0,
  "reasoning": "The Copilot-generated code successfully addresses all syntax and logical errors and enhances readability while adhering to Python coding standards. It maintains the functional accuracy of the original code."
}
data1 = {
  "table_data": [
    {
      "Criteria": "Correct arithmetic operation in add_numbers function",
      "Original Code Snippet": "return a - b",
      "Original Code Line Number": 2,
      "Copilot-Generated Code Snippet": "return a + b",
      "Copilot Code Line Number": 2,
      "Status": "Pass",
      "Explanation": "The subtraction operation in the original code was replaced with addition, correctly fixing the intended logical error in the function."
    },
    {
      "Criteria": "Logical implementation of print_numbers function",
      "Original Code Snippet": "while i < n:",
      "Original Code Line Number": 5,
      "Copilot-Generated Code Snippet": "while i >= 0:",
      "Copilot Code Line Number": 5,
      "Status": "Fail",
      "Explanation": "The condition while i >= 0 creates an infinite loop. This does not fix the logical error present in the original code, where decrementing i would lead to a faulty loop."
    },
    {
      "Criteria": "Increment/decrement logic in print_numbers function",
      "Original Code Snippet": "i -= 1",
      "Original Code Line Number": 7,
      "Copilot-Generated Code Snippet": "i += 1",
      "Copilot Code Line Number": 7,
      "Status": "Fail",
      "Explanation": "The counter change in the Copilot-generated code was incorrectly set to increment, which creates an infinite loop instead of properly iterating within bounds."
    },
    {
      "Criteria": "Correctness of check_even_odd function",
      "Original Code Snippet": "return 'Odd'",
      "Original Code Line Number": 11,
      "Copilot-Generated Code Snippet": "return 'Even'",
      "Copilot Code Line Number": 11,
      "Status": "Pass",
      "Explanation": "The logical error in the original code's check_even_odd function was fixed by swapping the return values for even and odd cases."
    }
  ],
  "score": 0.5,
  "reasoning": "While the arithmetic operation in the add_numbers function and the check_even_odd function's return values were corrected successfully, the changes to the print_numbers function introduced a new logical error—an infinite loop—by incorrectly altering the loop condition and iteration mechanism. Hence, only partial functionality was implemented as requested."
}
data = {
  "table_data": [
    {
      "Criteria": "Error Handling in Exception Block",
      "Original Code Snippet": "except:\n    print(\"except\")\n    logfile.writelines(...)\n    break",
      "Original Code Line Number": "33-37",
      "Copilot-Generated Code Snippet": "except Exception as e:\n    print(\"Exception:\", e)\n    logfile.writelines(...)\n    break",
      "Copilot Code Line Number": "28-32",
      "Status": "Pass",
      "Explanation": "The Copilot-generated code includes printing the specific exception message, which aids in debugging compared to the generic exception handling in the original code."
    },
    {
      "Criteria": "Input Documentation",
      "Original Code Snippet": "inputs:\n    roi_key: poi name\n    city_str: city name\n    baidu_ak: baidu web API AK\n    output: file save path",
      "Original Code Line Number": "4-8",
      "Copilot-Generated Code Snippet": "No equivalent input documentation provided.",
      "Copilot Code Line Number": "None",
      "Status": "Fail",
      "Explanation": "The Copilot-generated code omits input documentation entirely, which reduces code readability and maintainability."
    },
    {
      "Criteria": "Pagination Handling",
      "Original Code Snippet": "page_num += 1\ntime.sleep(1)",
      "Original Code Line Number": "29-30",
      "Copilot-Generated Code Snippet": "page_num += 1\ntime.sleep(1)",
      "Copilot Code Line Number": "25-26",
      "Status": "Pass",
      "Explanation": "The pagination logic is retained without modification and works correctly to increment page numbers for API requests."
    },
    {
      "Criteria": "File Closing",
      "Original Code Snippet": "No explicit file closure is implemented.",
      "Original Code Line Number": "None",
      "Copilot-Generated Code Snippet": "logfile.close()\nfile.close()",
      "Copilot Code Line Number": "33-34",
      "Status": "Pass",
      "Explanation": "Unlike the original code, the Copilot-generated version explicitly closes the open files, avoiding potential resource leakage."
    },
    {
      "Criteria": "Log Writing for Empty Results",
      "Original Code Snippet": "logfile.writelines(time.strftime(\"%Y-%m-%d-%H-%M-%S\") + \" \" + city_str + \" \" + str(page_num) + \"\\n\")",
      "Original Code Line Number": "22",
      "Copilot-Generated Code Snippet": "logfile.writelines(time.strftime(\"%Y-%m-%d-%H-%M-%S\") + \" \" + city_str + \" \" + str(page_num) + \"\\n\")",
      "Copilot Code Line Number": "17",
      "Status": "Pass",
      "Explanation": "The logic for logging empty results is preserved correctly, ensuring that missing data is recorded."
    },
    {
      "Criteria": "URL Construction",
      "Original Code Snippet": "URL = \"http://api.map.baidu.com/place/v2/search?query=\" + roi_key +\n    \"&region=\" + city_str +\n    \"&output=json\" +\n    \"&ak=\" + baidu_ak +\n    \"&scope=2\" +\n    \"&page_size=20\" +\n    \"&page_num=\" + str(page_num)",
      "Original Code Line Number": "13-19",
      "Copilot-Generated Code Snippet": "URL = \"http://api.map.baidu.com/place/v2/search?query=\" + roi_key + \"&region=\" + city_str + \"&output=json\" + \"&ak=\" + baidu_ak + \"&scope=2\" + \"&page_size=20\" + \"&page_num=\" + str(page_num)",
      "Copilot Code Line Number": "11-12",
      "Status": "Pass",
      "Explanation": "The URL construction remains functionally identical and ensures the API endpoint is correctly formed."
    },
    {
      "Criteria": "Response Handling",
      "Original Code Snippet": "res = json.loads(resp.text)\nif len(res[\"results\"]) == 0:",
      "Original Code Line Number": "20-21",
      "Copilot-Generated Code Snippet": "res = json.loads(resp.text)\nif len(res[\"results\"]) == 0:",
      "Copilot Code Line Number": "15-16",
      "Status": "Pass",
      "Explanation": "The response handling logic is preserved as in the original code, ensuring correct processing of API responses."
    }
  ],
  "score": 0.85,
  "reasoning": "The Copilot-generated code provides significant improvements, particularly in exception handling and resource management. However, it fails to document input parameters, which affects maintainability and completeness."
}
# Convert JSON to Pandas DataFrame
df = pd.DataFrame(data4["table_data"])

# Save DataFrame as CSV
csv_filename = "funcaccuracy_results_sample.csv"
df.to_csv(csv_filename, index=False)

# Display DataFrame inside Jupyter Notebook
print(f"CSV file '{csv_filename}' has been created successfully!")
display(df)


CSV file 'funcaccuracy_results_sample.csv' has been created successfully!


Functionality  \
0                  Generate a palindrome of length 5   
1  Use the middle character as the center of the ...   
2  Construct a string as 'first_two' + 'middle' +...   
3                    Output the generated palindrome   

                      Copilot-Generated Code Snippet Copilot Code Line Number  \
0  letters = "abcdefghijklmnopqrstuvwxyz"\nfirst_...                      3-6   
1                    middle = random.choice(letters)                        5   
2  palindrome = first_two + middle + first_two[::-1]                        6   
3  print("Generated Palindrome:", generate_palind...                        8   

  Status                                        Explanation  
0   Pass  The code correctly implements the functionalit...  
1   Pass  The middle character is correctly extracted as...  
2   Pass  The final palindrome string is constructed cor...  
3   Pass  The generated palindrome is output to the cons...

In [23]:
####cove#####
import os
import logging
from openai import AzureOpenAI

# Configuration management using environment variables
api_base = os.getenv("API_BASE", "https://tcoeaiteamgpt4o.openai.azure.com/")
api_key = os.getenv("API_KEY", "DH8RAnW9UnbzuSxm8luI7Go8nX42bdLhFIEKcPQ1U0JpOU0hPeLKJQQJ99BAACYeBjFXJ3w3AAABACOGdstk")
deployment_name = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = os.getenv("API_VERSION", "2024-05-01-preview")

# Initialize the AzureOpenAI client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}",
)

# Set up logging
# logging.basicConfig(level=logging.INFO)


def gpt4o_text_analyzer(user_prompt, prompt, response, golden_response):
    """
    Analyzes text using the GPT-4 model.

    Parameters:
    user_prompt (str): The user's prompt.
    prompt (str): The prompt to guide the text analysis.
    response (str): The Copilot generated code response.
    golden response (str):  The actual code response.

    Returns:
    str: The analysis result from the model.
    """
    # Parameter validation
    if not isinstance(user_prompt, str) or not user_prompt:
        raise ValueError("User prompt must be a non-empty string.")
    if not isinstance(prompt, str) or not prompt:
        raise ValueError("Prompt must be a non-empty string.")
    if not isinstance(response, str) or not response:
        raise ValueError("Response must be a non-empty string.")
    if not isinstance(golden_response, str) or not golden_response:
        raise ValueError("Golden response must be a non-empty string.")

    # logging.info(f"Analyzing text with user prompt: {user_prompt}, prompt: {prompt}, response: {response}, "
                 # f"and golden response: {golden_response}")
    try:
        messages = [
            {"role": "system",
             "content": "You are an expert text analyzing model that can understand and analyze text end to end."},
            {"role": "user", "content": user_prompt},
            {"role": "user", "content": prompt},
            {"role": "user", "content": response},
            {"role": "user", "content": golden_response}
        ]

        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=2000
        )
        # logging.info(f"Response: {response.choices[0].message.content}")
        return str(response.choices[0].message.content)
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return None


def gpt4o_cove(user_prompt, response, golden_response):
    cove_prompt = f'''You are provided with an original code{golden_response}, a user prompt{user_prompt}, and the corresponding Copilot-generated code{response}. Your task is to:

Generate Questions:

Create five hard-level questions that test the Copilot-generated code.
The questions should evaluate the code’s handling of edge cases, error management, logic consistency, and adherence to the user prompt.
Ensure the questions require detailed technical analysis and a deep comparison with the original code. Strictly DO NOT generate questions that are not related to the code.
Answer Questions:

Use the original code to answer the questions you created.
Verify whether the Copilot-generated code meets the requirements and solves the issues raised in the questions.
Scoring:

Calculate the overall score as the number of correct answers divided by the total number of questions asked.
Scale the score between 0 and 1.
Provide Output in JSON Format strictly:
The output should be strictly structured as follows:

"table_data": A list of dictionaries, where each dictionary contains the following keys question,answer and status:
"question": The generated question.
"answer": The answer derived from the original code.
"status": "Correct" if the answer aligns with the original code, otherwise "Incorrect".
"score": A float value between 0 and 1 representing the accuracy of Copilot-generated code.
"reasoning": A two-sentence summary explaining the reasoning behind the final score.'''

    
    response = gpt4o_text_analyzer(cove_prompt, user_prompt, response, golden_response)

    if response is None:
        return {"error": "Failed to analyze text for functional accuracy."}


    return response



user_prompt = ('''convert the code to python''')

response3 = '''IDENTIFICATION DIVISION.
PROGRAM-ID. INSERT-RECORD.
 
ENVIRONMENT DIVISION.
 
DATA DIVISION.
WORKING-STORAGE SECTION.
01  WS-STATUS-FLAGS.
    05 WS-DB-STATUS     PIC X(2).
       88 WS-SUCCESS    VALUE "00".
    05 WS-SQLCODE       PIC S9(9) COMP.
    05 WS-ERROR-MSG     PIC X(50).
 
LINKAGE SECTION.
01  LS-PARAMETERS.
    05 LS-PERSON-RECORD.
       10 PERSON-ID     PIC 9(6).
       10 PERSON-NAME   PIC X(50).
       10 PERSON-AGE    PIC 9(3).
    05 LS-RESULT        PIC X.
       88 SUCCESS       VALUE 'T'.
       88 FAILED        VALUE 'F'.
 
PROCEDURE DIVISION USING LS-PARAMETERS.
    PERFORM INSERT-AND-VALIDATE
    GOBACK
    .
 
INSERT-AND-VALIDATE.
    EXEC SQL
        INSERT INTO persons (id, name, age)
        VALUES (:PERSON-ID, :PERSON-NAME, :PERSON-AGE)
    END-EXEC
 
    IF SQLCODE = 0
        EXEC SQL COMMIT END-EXEC
        SET SUCCESS TO TRUE
    ELSE
        EXEC SQL ROLLBACK END-EXEC
        SET FAILED TO TRUE
        STRING "DB Error: " SQLCODE
            DELIMITED BY SIZE
            INTO WS-ERROR-MSG
        DISPLAY WS-ERROR-MSG
    END-IF
    .'''

response1 = '''import requests
import json
import time

# call API
def get_baidu_poi(roi_key, city_str, baidu_ak, output):
    now_time = time.strftime("%Y-%m-%d")
    page_num = 0
    logfile = open(output + "/" + now_time + ".log", "a+", encoding="utf-8")
    file = open(output + "/" + now_time + ".txt", "a+", encoding="utf-8")
    while True:
        try:
            URL = "http://api.map.baidu.com/place/v2/search?query=" + roi_key + "&region=" + city_str + "&output=json" + "&ak=" + baidu_ak + "&scope=2" + "&page_size=20" + "&page_num=" + str(page_num)
            resp = requests.get(URL)
            res = json.loads(resp.text)
            if len(res["results"]) == 0:
                logfile.writelines(time.strftime("%Y-%m-%d-%H-%M-%S") + " " + city_str + " " + str(page_num) + "\n")
                break
            else:
                for r in res["results"]:
                    j_name = r["name"]
                    j_lat = r["location"]["lat"]
                    j_lon = r["location"]["lng"]
                    j_area = r["area"]
                    j_add = r["address"]
                    j_str = str(j_name) + "," + str(j_lon) + "," + str(j_lat) + "," + str(j_area) + "," + str(j_add) + "\n"
                    file.writelines(j_str)
            page_num += 1
            time.sleep(1)
        except Exception as e:
            print("Exception:", e)
            logfile.writelines(time.strftime("%Y-%m-%d-%H-%M-%S") + " " + city_str + " " + str(page_num) + "\n")
            break
    logfile.close()
    file.close()'''

golden_response_2 = '''import pandas as pd
from pandas.io.formats.style import Styler
 
def process_data(item, price):
    # Cleanse data
    item = item.strip()  # Strip whitespace from item
    price = price.strip()  # Strip whitespace from price
    price = float(price) # Convert price to a float
    # More cleansing operations here
 
    # Create and print a DataFrame
    data = {'Item': [item], 'Price': [price]}
    df = pd.DataFrame(data)
    print(df.to_string(index=False))
 
# Example usage
item = "   Apple "
price = " 1.25"
process_data(item, price)'''

response = '''# IDENTIFICATION DIVISION = module/file name
# PROGRAM-ID = function name
def insert_record():
    pass
 
# DATA DIVISION would be like declaring these variables:
class WorkingStorage:
    db_status = ""  # WS-DB-STATUS
    sqlcode = 0     # WS-SQLCODE
    error_msg = ""  # WS-ERROR-MSG
 
# LINKAGE SECTION is like function parameters
class PersonRecord:
    def __init__(self, person_id: int, person_name: str, person_age: int):
        self.person_id = person_id    # PERSON-ID
        self.person_name = person_name # PERSON-NAME
        self.person_age = person_age   # PERSON-AGE
        self.success = False          # LS-RESULT'''

response1 = '''def cleanse_data(item, price):
  # Cleanse data
  item = item.strip()  # Strip whitespace from item
  price = price.strip()  # Strip whitespace from price
  price = float(price)  # Convert price to a float
  return item, price
 
def print_data(item, price):
  # Create and print a DataFrame
  data = {'Item': [item], 'Price': [price]}
  df = pd.DataFrame(data)
  print(df.to_string(index=False))
 
def process_data(item, price):
  item, price = cleanse_data(item, price)
  print_data(item, price)'''

golden_response = '''IDENTIFICATION DIVISION.
PROGRAM-ID. INSERT-RECORD.
 
ENVIRONMENT DIVISION.
 
DATA DIVISION.
WORKING-STORAGE SECTION.
01  WS-STATUS-FLAGS.
    05 WS-DB-STATUS     PIC X(2).
       88 WS-SUCCESS    VALUE "00".
    05 WS-SQLCODE       PIC S9(9) COMP.
    05 WS-ERROR-MSG     PIC X(50).
 
LINKAGE SECTION.
01  LS-PARAMETERS.
    05 LS-PERSON-RECORD.
       10 PERSON-ID     PIC 9(6).
       10 PERSON-NAME   PIC X(50).
       10 PERSON-AGE    PIC 9(3).
    05 LS-RESULT        PIC X.
       88 SUCCESS       VALUE 'T'.
       88 FAILED        VALUE 'F'.
 
PROCEDURE DIVISION USING LS-PARAMETERS.
    PERFORM INSERT-AND-VALIDATE
    GOBACK
    .
 
INSERT-AND-VALIDATE.
    EXEC SQL
        INSERT INTO persons (id, name, age)
        VALUES (:PERSON-ID, :PERSON-NAME, :PERSON-AGE)
    END-EXEC
 
    IF SQLCODE = 0
        EXEC SQL COMMIT END-EXEC
        SET SUCCESS TO TRUE
    ELSE
        EXEC SQL ROLLBACK END-EXEC
        SET FAILED TO TRUE
        STRING "DB Error: " SQLCODE
            DELIMITED BY SIZE
            INTO WS-ERROR-MSG
        DISPLAY WS-ERROR-MSG
    END-IF
    .'''

golden_response1 = '''import requests
import json
import time

# call API
def get_baidu_poi(roi_key, city_str, baidu_ak, output):
    """"""
    inputs:
        roi_key: poi name
        city_str: city name
        baidu_ak: baidu web API AK
        output: file save path
    """"""
    now_time = time.strftime(""%Y-%m-%d"")
    page_num = 0
    logfile = open(output + ""/"" + now_time + "".log"", ""a+"", encoding=""utf-8"")
    file = open(output + ""/"" + now_time + "".txt"", ""a+"", encoding=""utf-8"")
    while True:
        try:
            URL = ""http://api.map.baidu.com/place/v2/search?query="" + roi_key + \
                ""&region="" + city_str + \
                ""&output=json"" +  \
                ""&ak="" + baidu_ak + \
                ""&scope=2"" + \
                ""&page_size=20"" + \
                ""&page_num="" + str(page_num)
            resp = requests.get(URL)
            res = json.loads(resp.text)
            if len(res[""results""]) == 0:
                logfile.writelines(time.strftime(""%Y-%m-%d-%H-%M-%S"") + "" "" + city_str + "" "" + str(page_num) + ""\n"")
                break
            else:
                for r in res[""results""]:
                    j_name = r[""name""]
                    j_lat = r[""location""][""lat""]
                    j_lon = r[""location""][""lng""]
                    j_area = r[""area""]
                    j_add = r[""address""]
                    j_str = str(j_name) + "","" + str(j_lon) + "","" + str(j_lat) + "","" + str(j_area) + "","" + str(j_add) + ""\n""
                    file.writelines(j_str)
            page_num += 1
            time.sleep(1)
        except:
            print(""except"")
            logfile.writelines(time.strftime(""%Y-%m-%d-%H-%M-%S"") + "" "" + city_str + "" "" + str(page_num) + ""\n"")
            break'''




# Evaluate and print results for all four metrics
result = gpt4o_cove(user_prompt, response, golden_response)
print("cove Result:", result)

cove Result: Here’s the Python equivalent of the COBOL code you provided. The Python code uses a simple database insertion approach, simulating the logic in the original COBOL code:

```python
import sqlite3

class DatabaseManager:
    def __init__(self, db_name):
        self.connection = sqlite3.connect(db_name)
        self.cursor = self.connection.cursor()
        self.create_table()

    def create_table(self):
        # Create a table to mimic the "persons" table in your COBOL logic
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS persons (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            age INTEGER NOT NULL
        )
        """)
        self.connection.commit()

    def insert_record(self, person_id, person_name, person_age):
        db_status = None
        error_msg = ""
        result = 'F'

        try:
            # Insert the record into the database
            self.cursor.execute("INSERT INTO persons (id, name, age) VALU

In [22]:
import pandas as pd

# JSON Data
data1 = {
  "table_data": [
    {
      "question": "Does the Copilot-generated code correctly preserve the functionality of stripping whitespace and converting `price` to a float as specified in the original code?",
      "answer": "Yes, it correctly strips whitespace from both `item` and `price`, then converts `price` to a float.",
      "status": "Correct"
    },
    {
      "question": "Does the Copilot-generated code maintain the single responsibility principle by appropriately separating data cleansing and data printing into distinct functions?",
      "answer": "Yes, the Copilot-generated code separates the data cleansing and printing functions into `cleanse_data` and `print_data`, improving readability and modularity.",
      "status": "Correct"
    },
    {
      "question": "How does the Copilot-generated code handle scenarios where `item` or `price` is empty or `None`?",
      "answer": "The Copilot-generated code does not add error handling for empty or `None` values, which may lead to runtime exceptions similar to the original code.",
      "status": "Incorrect"
    },
    {
      "question": "In the Copilot-generated code, is the flow of execution for cleansing and printing data consistent with the intent of the original code?",
      "answer": "Yes, the flow of execution is consistent with the original code. However, the abstraction makes the code more organized and flexible for future use cases.",
      "status": "Correct"
    },
    {
      "question": "If additional cleansing operations were added in the future, does the Copilot restructuring provide better extensibility for maintaining and updating the code compared to the original implementation?",
      "answer": "Yes, the Copilot restructuring makes it more modular and easier to extend compared to the original implementation.",
      "status": "Correct"
    }
  ],
  "score": 0.8,
  "reasoning": "The Copilot-generated code improved modularity and readability by separating responsibilities between functions. However, both the original and Copilot-generated codes lack error handling for missing or invalid input values."
}
data = {
  "table_data": [
    {
      "question": "Does the Copilot-generated code handle exceptions properly by providing a meaningful error message and logging it?",
      "answer": "The original code does not provide a meaningful error message in the exception block; it only prints 'except' and logs a generic message. The Copilot-generated code improves this by including the exception details in the log (using `Exception as e`).",
      "status": "Correct"
    },
    {
      "question": "Does the Copilot-generated code correctly ensure that files (`logfile` and `file`) are closed at the end of function execution?",
      "answer": "The original code does not ensure that the `logfile` and `file` objects are closed explicitly. The Copilot-generated code closes these files explicitly at the end of the function using `logfile.close()` and `file.close()`.",
      "status": "Correct"
    },
    {
      "question": "Does the Copilot-generated code handle API responses accurately and ensure that empty results are processed correctly?",
      "answer": "The original code logs a message and breaks the loop when empty results are detected. The Copilot-generated code replicates this behavior accurately, ensuring no infinite loop occurs when all results are fetched.",
      "status": "Correct"
    },
    {
      "question": "Does the Copilot-generated code adhere to logging standards and log the city name, current page number, and timestamp consistently?",
      "answer": "The original code logs these details in the file, providing a timestamp, city name, and page number. The Copilot-generated code adheres to the same logging structure, maintaining consistency with the original implementation.",
      "status": "Correct"
    },
    {
      "question": "Does the Copilot-generated code ensure that the Baidu API endpoint and parameters are constructed correctly?",
      "answer": "Both the original and Copilot-generated code construct the Baidu API URL correctly with necessary parameters such as `query`, `region`, `output`, `ak`, `scope`, `page_size`, and `page_num`. There is no issue with the construction of the API endpoint.",
      "status": "Correct"
    }
  ],
  "score": 1.0,
  "reasoning": "The Copilot-generated code successfully resolves issues present in the original code, particularly in exception handling, file handling, and adherence to logging and API response requirements. All five questions were correctly addressed, leading to a perfect score."
}
# Convert JSON to Pandas DataFrame
df = pd.DataFrame(data1["table_data"])

# Save DataFrame as CSV
csv_filename = "cove_results_2.csv"
df.to_csv(csv_filename, index=False)

# Display DataFrame inside Jupyter Notebook
print(f"CSV file '{csv_filename}' has been created successfully!")
display(df)


CSV file 'cove_results_2.csv' has been created successfully!


question  \
0  Does the Copilot-generated code correctly pres...   
1  Does the Copilot-generated code maintain the s...   
2  How does the Copilot-generated code handle sce...   
3  In the Copilot-generated code, is the flow of ...   
4  If additional cleansing operations were added ...   

                                              answer     status  
0  Yes, it correctly strips whitespace from both ...    Correct  
1  Yes, the Copilot-generated code separates the ...    Correct  
2  The Copilot-generated code does not add error ...  Incorrect  
3  Yes, the flow of execution is consistent with ...    Correct  
4  Yes, the Copilot restructuring makes it more m...    Correct

In [26]:
######### instruction handling #############

import os
import logging
from openai import AzureOpenAI

# Configuration management using environment variables
api_base = os.getenv("API_BASE", "https://tcoeaiteamgpt4o.openai.azure.com/")
api_key = os.getenv("API_KEY", "DH8RAnW9UnbzuSxm8luI7Go8nX42bdLhFIEKcPQ1U0JpOU0hPeLKJQQJ99BAACYeBjFXJ3w3AAABACOGdstk")
deployment_name = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = os.getenv("API_VERSION", "2024-05-01-preview")

# Initialize the AzureOpenAI client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}",
)

# Set up logging
# logging.basicConfig(level=logging.INFO)


def gpt4o_text_analyzer(user_prompt, prompt, response, golden_response):
    """
    Analyzes text using the GPT-4 model.

    Parameters:
    user_prompt (str): The user's prompt.
    prompt (str): The prompt to guide the text analysis.
    response (str): The Copilot generated code response.
    golden response (str):  The actual code response.

    Returns:
    str: The analysis result from the model.
    """
    # Parameter validation
    if not isinstance(user_prompt, str) or not user_prompt:
        raise ValueError("User prompt must be a non-empty string.")
    if not isinstance(prompt, str) or not prompt:
        raise ValueError("Prompt must be a non-empty string.")
    if not isinstance(response, str) or not response:
        raise ValueError("Response must be a non-empty string.")
    if not isinstance(golden_response, str) or not golden_response:
        raise ValueError("Golden response must be a non-empty string.")

    # logging.info(f"Analyzing text with user prompt: {user_prompt}, prompt: {prompt}, response: {response}, "
                 # f"and golden response: {golden_response}")
    try:
        messages = [
            {"role": "system",
             "content": "You are an expert text analyzing model that can understand and analyze text end to end."},
            {"role": "user", "content": user_prompt},
            {"role": "user", "content": prompt},
            {"role": "user", "content": response},
            {"role": "user", "content": golden_response}
        ]

        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=2000
        )
        # logging.info(f"Response: {response.choices[0].message.content}")
        return str(response.choices[0].message.content)
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return None


def gpt4o_instruction_handling(user_prompt, response, golden_response):
    instructionhandler_prompt = f'''You are provided with an original code{golden_response}, a user prompt{user_prompt}, and the corresponding Copilot-generated code{response}. Your task is to:

Instructions:
Extract Instructions:

Identify only the niche instruction from the user prompt. Focus only on the main specific instructions in the prompt and not all the general instructions.
Break them down into distinct, actionable tasks that the Copilot-generated code is expected to implement.
Compare with Copilot-Generated Code:

Locate the specific code modifications made by Copilot that correspond to each instruction.
Record the modified code snippet along with its line number in the Copilot-generated code.
Evaluate Adherence to Instructions:

If the instruction has been correctly implemented, mark it as "instruction adhered".
If the instruction has not been followed or incorrectly implemented, mark it as "instruction not adhered".
Provide a brief reasoning for the status.
Scoring:

The score is calculated as:
score=total instructions given in prompt/
number of adhered instructions
Scale the score between 0 and 1.
Provide Output in JSON Format strictly with the keys tab;e_data,score and reasoning, the values of each key is instructed in the following manner:

"table_data": A list of dictionaries, where each dictionary contains the following keys "Niche Instruction in the prompt","Code modified","line number","status","Reasoning behind status":
"Niche Instruction in the prompt": The extracted instruction from the user prompt.
"Code modified": The corresponding modification in the Copilot-generated code.
"line number": The line number where the modification occurs.
"status": "instruction adhered" if correctly implemented, otherwise "instruction not adhered".
"Reasoning behind status": A brief explanation of why the instruction was or was not adhered to.
"score": A float value between 0 and 1, representing how well the Copilot-generated code followed the instructions.
"reasoning": A two-sentence summary explaining the final score.'''

    
    response = gpt4o_text_analyzer(instructionhandler_prompt, user_prompt, response, golden_response)

    if response is None:
        return {"error": "Failed to analyze text for functional accuracy."}


    return response



user_prompt = ('''Convert the code to python.Ensure it has proper exception handlers.Comments must be added for each and every conversion to python code. Ensure the pep8 coding standards are followed. Ensure the naming conventions are maintained properly''')

response = '''
def insert_record():
    pass
 
# DATA DIVISION would be like declaring these variables:
class WorkingStorage:
    db_status = ""  
    sqlcode = 0    
    error_msg = ""  
 
# LINKAGE SECTION is like function parameters
class PersonRecord:
    def __init__(self, person_id: int, person_name: str, person_age: int):
        self.person_id = person_id    
        self.person_name = person_name 
        self.person_age = person_age   
        self.success = False  '''

golden_response = '''IDENTIFICATION DIVISION.
PROGRAM-ID. INSERT-RECORD.
 
ENVIRONMENT DIVISION.
 
DATA DIVISION.
WORKING-STORAGE SECTION.
01  WS-STATUS-FLAGS.
    05 WS-DB-STATUS     PIC X(2).
       88 WS-SUCCESS    VALUE "00".
    05 WS-SQLCODE       PIC S9(9) COMP.
    05 WS-ERROR-MSG     PIC X(50).
 
LINKAGE SECTION.
01  LS-PARAMETERS.
    05 LS-PERSON-RECORD.
       10 PERSON-ID     PIC 9(6).
       10 PERSON-NAME   PIC X(50).
       10 PERSON-AGE    PIC 9(3).
    05 LS-RESULT        PIC X.
       88 SUCCESS       VALUE 'T'.
       88 FAILED        VALUE 'F'.
 
PROCEDURE DIVISION USING LS-PARAMETERS.
    PERFORM INSERT-AND-VALIDATE
    GOBACK
    .
 
INSERT-AND-VALIDATE.
    EXEC SQL
        INSERT INTO persons (id, name, age)
        VALUES (:PERSON-ID, :PERSON-NAME, :PERSON-AGE)
    END-EXEC
 
    IF SQLCODE = 0
        EXEC SQL COMMIT END-EXEC
        SET SUCCESS TO TRUE
    ELSE
        EXEC SQL ROLLBACK END-EXEC
        SET FAILED TO TRUE
        STRING "DB Error: " SQLCODE
            DELIMITED BY SIZE
            INTO WS-ERROR-MSG
        DISPLAY WS-ERROR-MSG
    END-IF
    .'''
response1 = '''import requests
import json
import time

# call API
def get_baidu_poi(roi_key, city_str, baidu_ak, output):
    now_time = time.strftime("%Y-%m-%d")
    page_num = 0
    logfile = open(output + "/" + now_time + ".log", "a+", encoding="utf-8")
    file = open(output + "/" + now_time + ".txt", "a+", encoding="utf-8")
    while True:
        try:
            URL = "http://api.map.baidu.com/place/v2/search?query=" + roi_key + "&region=" + city_str + "&output=json" + "&ak=" + baidu_ak + "&scope=2" + "&page_size=20" + "&page_num=" + str(page_num)
            resp = requests.get(URL)
            res = json.loads(resp.text)
            if len(res["results"]) == 0:
                logfile.writelines(time.strftime("%Y-%m-%d-%H-%M-%S") + " " + city_str + " " + str(page_num) + "\n")
                break
            else:
                for r in res["results"]:
                    j_name = r["name"]
                    j_lat = r["location"]["lat"]
                    j_lon = r["location"]["lng"]
                    j_area = r["area"]
                    j_add = r["address"]
                    j_str = str(j_name) + "," + str(j_lon) + "," + str(j_lat) + "," + str(j_area) + "," + str(j_add) + "\n"
                    file.writelines(j_str)
            page_num += 1
            time.sleep(1)
        except Exception as e:
            print("Exception:", e)
            logfile.writelines(time.strftime("%Y-%m-%d-%H-%M-%S") + " " + city_str + " " + str(page_num) + "\n")
            break
    logfile.close()
    file.close()'''


golden_response1 = '''import requests
import json
import time

# call API
def get_baidu_poi(roi_key, city_str, baidu_ak, output):
    """"""
    inputs:
        roi_key: poi name
        city_str: city name
        baidu_ak: baidu web API AK
        output: file save path
    """"""
    now_time = time.strftime(""%Y-%m-%d"")
    page_num = 0
    logfile = open(output + ""/"" + now_time + "".log"", ""a+"", encoding=""utf-8"")
    file = open(output + ""/"" + now_time + "".txt"", ""a+"", encoding=""utf-8"")
    while True:
        try:
            URL = ""http://api.map.baidu.com/place/v2/search?query="" + roi_key + \
                ""&region="" + city_str + \
                ""&output=json"" +  \
                ""&ak="" + baidu_ak + \
                ""&scope=2"" + \
                ""&page_size=20"" + \
                ""&page_num="" + str(page_num)
            resp = requests.get(URL)
            res = json.loads(resp.text)
            if len(res[""results""]) == 0:
                logfile.writelines(time.strftime(""%Y-%m-%d-%H-%M-%S"") + "" "" + city_str + "" "" + str(page_num) + ""\n"")
                break
            else:
                for r in res[""results""]:
                    j_name = r[""name""]
                    j_lat = r[""location""][""lat""]
                    j_lon = r[""location""][""lng""]
                    j_area = r[""area""]
                    j_add = r[""address""]
                    j_str = str(j_name) + "","" + str(j_lon) + "","" + str(j_lat) + "","" + str(j_area) + "","" + str(j_add) + ""\n""
                    file.writelines(j_str)
            page_num += 1
            time.sleep(1)
        except:
            print(""except"")
            logfile.writelines(time.strftime(""%Y-%m-%d-%H-%M-%S"") + "" "" + city_str + "" "" + str(page_num) + ""\n"")
            break'''




# Evaluate and print results for all four metrics
result = gpt4o_instruction_handling(user_prompt, response, golden_response)
print("Instuction Handling Result:", result)

Instuction Handling Result: ```json
{
  "table_data": [
    {
      "Niche Instruction in the prompt": "Convert the code to python.",
      "Code modified": "def insert_record():\n    pass",
      "line number": 1,
      "status": "instruction not adhered",
      "Reasoning behind status": "Code structure begins with a placeholder function, but it does not convert the COBOL logic into Python. Actual logic translation and SQL handling are missing."
    },
    {
      "Niche Instruction in the prompt": "Ensure it has proper exception handlers.",
      "Code modified": "No exception handling code is provided.",
      "line number": "N/A",
      "status": "instruction not adhered",
      "Reasoning behind status": "Exception handling for SQL operations or other errors was not implemented."
    },
    {
      "Niche Instruction in the prompt": "Comments must be added for each and every conversion to python code.",
      "Code modified": "# DATA DIVISION would be like declaring these variabl

In [ ]:
######### instruction handling #############

import os
import logging
from openai import AzureOpenAI

# Configuration management using environment variables
api_base = os.getenv("API_BASE", "https://tcoeaiteamgpt4o.openai.azure.com/")
api_key = os.getenv("API_KEY", "DH8RAnW9UnbzuSxm8luI7Go8nX42bdLhFIEKcPQ1U0JpOU0hPeLKJQQJ99BAACYeBjFXJ3w3AAABACOGdstk")
deployment_name = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = os.getenv("API_VERSION", "2024-05-01-preview")

# Initialize the AzureOpenAI client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}",
)

# Set up logging
# logging.basicConfig(level=logging.INFO)


def gpt4o_text_analyzer(user_prompt, prompt, response, golden_response):
    """
    Analyzes text using the GPT-4 model.

    Parameters:
    user_prompt (str): The user's prompt.
    prompt (str): The prompt to guide the text analysis.
    response (str): The Copilot generated code response.
    golden response (str):  The actual code response.

    Returns:
    str: The analysis result from the model.
    """
    # Parameter validation
    if not isinstance(user_prompt, str) or not user_prompt:
        raise ValueError("User prompt must be a non-empty string.")
    if not isinstance(prompt, str) or not prompt:
        raise ValueError("Prompt must be a non-empty string.")
    if not isinstance(response, str) or not response:
        raise ValueError("Response must be a non-empty string.")
    if not isinstance(golden_response, str) or not golden_response:
        raise ValueError("Golden response must be a non-empty string.")

    # logging.info(f"Analyzing text with user prompt: {user_prompt}, prompt: {prompt}, response: {response}, "
                 # f"and golden response: {golden_response}")
    try:
        messages = [
            {"role": "system",
             "content": "You are an expert text analyzing model that can understand and analyze text end to end."},
            {"role": "user", "content": user_prompt},
            {"role": "user", "content": prompt},
            {"role": "user", "content": response},
            {"role": "user", "content": golden_response}
        ]

        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=2000
        )
        # logging.info(f"Response: {response.choices[0].message.content}")
        return str(response.choices[0].message.content)
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return None


def gpt4o_instruction_handling(user_prompt, response, golden_response):
    instructionhandler_prompt = f'''You are provided with an original code{golden_response}, a user prompt{user_prompt}, and the corresponding Copilot-generated code{response}. Your task is to:

Instructions:
Extract Instructions:

Identify only the niche instruction from the user prompt. Focus only on the main specific instructions in the prompt and not all the general instructions.
Break them down into distinct, actionable tasks that the Copilot-generated code is expected to implement.
Compare with Copilot-Generated Code:

Locate the specific code modifications made by Copilot that correspond to each instruction.
Record the modified code snippet along with its line number in the Copilot-generated code.
Evaluate Adherence to Instructions:

If the instruction has been correctly implemented, mark it as "instruction adhered".
If the instruction has not been followed or incorrectly implemented, mark it as "instruction not adhered".
Provide a brief reasoning for the status.
Scoring:

The score is calculated as:
score=total instructions given in prompt/
number of adhered instructions
Scale the score between 0 and 1.
Provide Output in JSON Format strictly with the keys tab;e_data,score and reasoning, the values of each key is instructed in the following manner:

"table_data": A list of dictionaries, where each dictionary contains the following keys "Niche Instruction in the prompt","Code modified","line number","status","Reasoning behind status":
"Niche Instruction in the prompt": The extracted instruction from the user prompt.
"Code modified": The corresponding modification in the Copilot-generated code.
"line number": The line number where the modification occurs.
"status": "instruction adhered" if correctly implemented, otherwise "instruction not adhered".
"Reasoning behind status": A brief explanation of why the instruction was or was not adhered to.
"score": A float value between 0 and 1, representing how well the Copilot-generated code followed the instructions.
"reasoning": A two-sentence summary explaining the final score.'''

    
    response = gpt4o_text_analyzer(instructionhandler_prompt, user_prompt, response, golden_response)

    if response is None:
        return {"error": "Failed to analyze text for functional accuracy."}


    return response



user_prompt = ('''Write a python program to Generate a palindrome of length 5''')

response = '''import random

def generate_palindrome():
    letters = "abcdefghijklmnopqrstuvwxyz"
    first_two = "".join(random.choices(letters, k=2))
    middle = random.choice(letters)
    palindrome = first_two + middle + first_two[::-1]
    return palindrome

print("Generated Palindrome:", generate_palindrome())
'''

response4 = '''
def insert_record():
    pass
 
# DATA DIVISION would be like declaring these variables:
class WorkingStorage:
    db_status = ""  
    sqlcode = 0    
    error_msg = ""  
 
# LINKAGE SECTION is like function parameters
class PersonRecord:
    def __init__(self, person_id: int, person_name: str, person_age: int):
        self.person_id = person_id    
        self.person_name = person_name 
        self.person_age = person_age   
        self.success = False  '''

golden_response = '''IDENTIFICATION DIVISION.
PROGRAM-ID. INSERT-RECORD.
 
ENVIRONMENT DIVISION.
 
DATA DIVISION.
WORKING-STORAGE SECTION.
01  WS-STATUS-FLAGS.
    05 WS-DB-STATUS     PIC X(2).
       88 WS-SUCCESS    VALUE "00".
    05 WS-SQLCODE       PIC S9(9) COMP.
    05 WS-ERROR-MSG     PIC X(50).
 
LINKAGE SECTION.
01  LS-PARAMETERS.
    05 LS-PERSON-RECORD.
       10 PERSON-ID     PIC 9(6).
       10 PERSON-NAME   PIC X(50).
       10 PERSON-AGE    PIC 9(3).
    05 LS-RESULT        PIC X.
       88 SUCCESS       VALUE 'T'.
       88 FAILED        VALUE 'F'.
 
PROCEDURE DIVISION USING LS-PARAMETERS.
    PERFORM INSERT-AND-VALIDATE
    GOBACK
    .
 
INSERT-AND-VALIDATE.
    EXEC SQL
        INSERT INTO persons (id, name, age)
        VALUES (:PERSON-ID, :PERSON-NAME, :PERSON-AGE)
    END-EXEC
 
    IF SQLCODE = 0
        EXEC SQL COMMIT END-EXEC
        SET SUCCESS TO TRUE
    ELSE
        EXEC SQL ROLLBACK END-EXEC
        SET FAILED TO TRUE
        STRING "DB Error: " SQLCODE
            DELIMITED BY SIZE
            INTO WS-ERROR-MSG
        DISPLAY WS-ERROR-MSG
    END-IF
    .'''
response1 = '''import requests
import json
import time

# call API
def get_baidu_poi(roi_key, city_str, baidu_ak, output):
    now_time = time.strftime("%Y-%m-%d")
    page_num = 0
    logfile = open(output + "/" + now_time + ".log", "a+", encoding="utf-8")
    file = open(output + "/" + now_time + ".txt", "a+", encoding="utf-8")
    while True:
        try:
            URL = "http://api.map.baidu.com/place/v2/search?query=" + roi_key + "&region=" + city_str + "&output=json" + "&ak=" + baidu_ak + "&scope=2" + "&page_size=20" + "&page_num=" + str(page_num)
            resp = requests.get(URL)
            res = json.loads(resp.text)
            if len(res["results"]) == 0:
                logfile.writelines(time.strftime("%Y-%m-%d-%H-%M-%S") + " " + city_str + " " + str(page_num) + "\n")
                break
            else:
                for r in res["results"]:
                    j_name = r["name"]
                    j_lat = r["location"]["lat"]
                    j_lon = r["location"]["lng"]
                    j_area = r["area"]
                    j_add = r["address"]
                    j_str = str(j_name) + "," + str(j_lon) + "," + str(j_lat) + "," + str(j_area) + "," + str(j_add) + "\n"
                    file.writelines(j_str)
            page_num += 1
            time.sleep(1)
        except Exception as e:
            print("Exception:", e)
            logfile.writelines(time.strftime("%Y-%m-%d-%H-%M-%S") + " " + city_str + " " + str(page_num) + "\n")
            break
    logfile.close()
    file.close()'''


golden_response1 = '''import requests
import json
import time

# call API
def get_baidu_poi(roi_key, city_str, baidu_ak, output):
    """"""
    inputs:
        roi_key: poi name
        city_str: city name
        baidu_ak: baidu web API AK
        output: file save path
    """"""
    now_time = time.strftime(""%Y-%m-%d"")
    page_num = 0
    logfile = open(output + ""/"" + now_time + "".log"", ""a+"", encoding=""utf-8"")
    file = open(output + ""/"" + now_time + "".txt"", ""a+"", encoding=""utf-8"")
    while True:
        try:
            URL = ""http://api.map.baidu.com/place/v2/search?query="" + roi_key + \
                ""&region="" + city_str + \
                ""&output=json"" +  \
                ""&ak="" + baidu_ak + \
                ""&scope=2"" + \
                ""&page_size=20"" + \
                ""&page_num="" + str(page_num)
            resp = requests.get(URL)
            res = json.loads(resp.text)
            if len(res[""results""]) == 0:
                logfile.writelines(time.strftime(""%Y-%m-%d-%H-%M-%S"") + "" "" + city_str + "" "" + str(page_num) + ""\n"")
                break
            else:
                for r in res[""results""]:
                    j_name = r[""name""]
                    j_lat = r[""location""][""lat""]
                    j_lon = r[""location""][""lng""]
                    j_area = r[""area""]
                    j_add = r[""address""]
                    j_str = str(j_name) + "","" + str(j_lon) + "","" + str(j_lat) + "","" + str(j_area) + "","" + str(j_add) + ""\n""
                    file.writelines(j_str)
            page_num += 1
            time.sleep(1)
        except:
            print(""except"")
            logfile.writelines(time.strftime(""%Y-%m-%d-%H-%M-%S"") + "" "" + city_str + "" "" + str(page_num) + ""\n"")
            break'''




# Evaluate and print results for all four metrics
result = gpt4o_instruction_handling(user_prompt, response)
print("Instuction Handling Result:", result)

In [27]:
import pandas as pd

# JSON Data
data2 = {
  "table_data": [
    {
      "Niche Instruction in the prompt": "Convert the code to python.",
      "Code modified": "def insert_record():\n    pass",
      "line number": 1,
      "status": "instruction not adhered",
      "Reasoning behind status": "Code structure begins with a placeholder function, but it does not convert the COBOL logic into Python. Actual logic translation and SQL handling are missing."
    },
    {
      "Niche Instruction in the prompt": "Ensure it has proper exception handlers.",
      "Code modified": "No exception handling code is provided.",
      "line number": "N/A",
      "status": "instruction not adhered",
      "Reasoning behind status": "Exception handling for SQL operations or other errors was not implemented."
    },
    {
      "Niche Instruction in the prompt": "Comments must be added for each and every conversion to python code.",
      "Code modified": "# DATA DIVISION would be like declaring these variables:",
      "line number": 4,
      "status": "instruction adhered",
      "Reasoning behind status": "Comments explaining the correspondence between COBOL Data Division and Python variables are included."
    },
    {
      "Niche Instruction in the prompt": "Ensure the pep8 coding standards are followed.",
      "Code modified": "class WorkingStorage:\n    db_status = \"\"\n    sqlcode = 0\n    error_msg = \"\"",
      "line number": 5,
      "status": "instruction adhered",
      "Reasoning behind status": "The code follows proper Python indentation and PEP8 guidelines for class and variable declarations."
    },
    {
      "Niche Instruction in the prompt": "Ensure the naming conventions are maintained properly.",
      "Code modified": "class PersonRecord:\n    def __init__(self, person_id: int, person_name: str, person_age: int):\n        self.person_id = person_id\n        self.person_name = person_name\n        self.person_age = person_age\n        self.success = False",
      "line number": 11,
      "status": "instruction adhered",
      "Reasoning behind status": "The conversion preserves meaningful naming conventions from COBOL fields into Python's snake_case style, which aligns with PEP8."
    }
  ],
  "score": 0.6,
  "reasoning": "While naming conventions and comments were correctly implemented, the core logic conversion and exception handling were missing, significantly affecting the adherence score."
}
data1 = {
  "table_data": [
    {
      "Niche Instruction in the prompt": "Add proper comments in the code.",
      "Code modified": "No comments were added.",
      "line number": "None",
      "status": "instruction not adhered",
      "Reasoning behind status": "The Copilot-generated code includes no added comments explaining the function, logic, or structure, which was explicitly requested in the prompt."
    },
    {
      "Niche Instruction in the prompt": "Ensure the code correctly handles empty API responses.",
      "Code modified": "if len(res[\"results\"]) == 0: logfile.writelines(...) break",
      "line number": "17-19 in the Copilot-generated code",
      "status": "instruction adhered",
      "Reasoning behind status": "The Copilot-generated code has a condition to check if the results are empty and breaks the loop efficiently, also logs the required information."
    },
    {
      "Niche Instruction in the prompt": "Ensure the code correctly handles exceptions properly.",
      "Code modified": "except Exception as e: print(\"Exception:\", e) logfile.writelines(...) break",
      "line number": "27-30 in the Copilot-generated code",
      "status": "instruction adhered",
      "Reasoning behind status": "An exception block has been added, displaying the exception message and logging it, which fulfills the requirement."
    },
    {
      "Niche Instruction in the prompt": "Fix existing issues in the flow so it can fetch POI data from the Baidu API without errors.",
      "Code modified": "Copilot made no logic fixes to directly address errors like failed API calls or data inconsistencies.",
      "line number": "None",
      "status": "instruction not adhered",
      "Reasoning behind status": "Apart from handling empty responses and exceptions, the logic for ensuring proper API calls (e.g., verifying status codes) or data format validation was not implemented."
    }
  ],
  "score": 0.5,
  "reasoning": "While the Copilot-generated code added handling for empty results and exceptions, it failed to add the requested comments and adequately address all potential issues in API response handling and flow."
}
data = {
  "table_data": [
    {
      "instruction in the prompt": "The code must handle API responses or flow correctly.",
      "Code modified": "Exception handling improved with `Exception as e` to explicitly capture and log the exception details.",
      "line number": "24-27",
      "status": "instruction adhered",
      "Reasoning behind status": "The copilot-generated code introduced an explicit exception catch with details logged, ensuring better handling of unexpected scenarios during API calls."
    },
    {
      "instruction in the prompt": "Ensure it can fetch POI data from the Baidu API without any errors.",
      "Code modified": "Properly checks for the presence of results using `if len(res['results']) == 0` and exits the loop when no results are present.",
      "line number": "13-16",
      "status": "instruction adhered",
      "Reasoning behind status": "The code correctly exits the loop if there are no more results, ensuring that API calls handle empty responses gracefully."
    },
    {
      "instruction in the prompt": "Ensure it handles empty results properly.",
      "Code modified": "Logs empty results with relevant details to a log file using `logfile.writelines(...)`.",
      "line number": "14-15",
      "status": "instruction adhered",
      "Reasoning behind status": "The code logs the details of empty results properly, indicating the scenario is being handled as intended."
    },
    {
      "instruction in the prompt": "Ensure it handles exceptions properly.",
      "Code modified": "Introduced `Exception as e` and prints/logs the exception message.",
      "line number": "24-27",
      "status": "instruction adhered",
      "Reasoning behind status": "The exception handling now provides a clear indication of what went wrong, adhering to the requirement to manage errors effectively."
    }
  ]
}

# Convert JSON to Pandas DataFrame
df = pd.DataFrame(data2["table_data"])

# Save DataFrame as CSV
csv_filename = "instruction_handling_results_2.csv"
df.to_csv(csv_filename, index=False)

# Display DataFrame inside Jupyter Notebook
print(f"CSV file '{csv_filename}' has been created successfully!")
display(df)


CSV file 'instruction_handling_results_2.csv' has been created successfully!


Niche Instruction in the prompt  \
0                        Convert the code to python.   
1           Ensure it has proper exception handlers.   
2  Comments must be added for each and every conv...   
3     Ensure the pep8 coding standards are followed.   
4  Ensure the naming conventions are maintained p...   

                                       Code modified line number  \
0                     def insert_record():\n    pass           1   
1            No exception handling code is provided.         N/A   
2  # DATA DIVISION would be like declaring these ...           4   
3  class WorkingStorage:\n    db_status = ""\n   ...           5   
4  class PersonRecord:\n    def __init__(self, pe...          11   

                    status                            Reasoning behind status  
0  instruction not adhered  Code structure begins with a placeholder funct...  
1  instruction not adhered  Exception handling for SQL operations or other...  
2      instruction adhered  Comments explaining the correspondence between...  
3      instruction adhered  The code follows proper Python indentation and...  
4      instruction adhered  The conversion preserves meaningful naming con...

In [17]:
######### Errors #############

import os
import logging
from openai import AzureOpenAI

# Configuration management using environment variables
api_base = os.getenv("API_BASE", "https://tcoeaiteamgpt4o.openai.azure.com/")
api_key = os.getenv("API_KEY", "DH8RAnW9UnbzuSxm8luI7Go8nX42bdLhFIEKcPQ1U0JpOU0hPeLKJQQJ99BAACYeBjFXJ3w3AAABACOGdstk")
deployment_name = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = os.getenv("API_VERSION", "2024-05-01-preview")

# Initialize the AzureOpenAI client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}",
)

# Set up logging
# logging.basicConfig(level=logging.INFO)


def gpt4o_text_analyzer(user_prompt, prompt, response, golden_response):
    """
    Analyzes text using the GPT-4 model.

    Parameters:
    user_prompt (str): The user's prompt.
    prompt (str): The prompt to guide the text analysis.
    response (str): The Copilot generated code response.
    golden response (str):  The actual code response.

    Returns:
    str: The analysis result from the model.
    """
    # Parameter validation
    if not isinstance(user_prompt, str) or not user_prompt:
        raise ValueError("User prompt must be a non-empty string.")
    if not isinstance(prompt, str) or not prompt:
        raise ValueError("Prompt must be a non-empty string.")
    if not isinstance(response, str) or not response:
        raise ValueError("Response must be a non-empty string.")
    if not isinstance(golden_response, str) or not golden_response:
        raise ValueError("Golden response must be a non-empty string.")

    # logging.info(f"Analyzing text with user prompt: {user_prompt}, prompt: {prompt}, response: {response}, "
                 # f"and golden response: {golden_response}")
    try:
        messages = [
            {"role": "system",
             "content": "You are an expert text analyzing model that can understand and analyze text end to end."},
            {"role": "user", "content": user_prompt},
            {"role": "user", "content": prompt},
            {"role": "user", "content": response},
            {"role": "user", "content": golden_response}
        ]

        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=2000
        )
        # logging.info(f"Response: {response.choices[0].message.content}")
        return str(response.choices[0].message.content)
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return None


def gpt4o_error(user_prompt, response, golden_response):
    error_prompt = f'''You are provided with an original code{golden_response}, a user prompt{user_prompt}, and the corresponding Copilot-generated code{response}. 

you need to acts as a error spotting agent. Your goal is to check if there are any errors in the copilot genertaed code.
The original code may have errors but that has to be fixed in the copilot genertaed code, you need to first check if that error is fixed in the copilot genertaed code and also check if new errors are introduced in he copilot code.
Identify each and every error such as syntax error, logical error etc.
Scoring:Scale the score between 0 and 1. If an error is not fixed then significantly reduce the score.
Provide Output in JSON Format strictly with the keys table_data,score and reasoning, the values of each key is instructed in the following manner:

"table_data": A list of dictionaries, where each dictionary contains the following keys "Error Identified in the Copilot code","line number of the error in the code","Is New error/old Error":
"Error Identified in the Copilot code": Identify the errors in the copilot code.
"line number of the error in the code": The line number where the error occurs.
"Is New error/old Error": 'New Error' if it is a newly introduced error by the copilot code , 'old Error' otherwise .
"score": A float value between 0 and 1, representing the number of errors generated.
"reasoning": A two-sentence summary explaining the final score.'''

    
    response = gpt4o_text_analyzer(error_prompt, user_prompt, response, golden_response)

    if response is None:
        return {"error": "Failed to analyze text."}


    return response



user_prompt = ('''There seems to be an issue in the code where it might not be handling API responses or the flow correctly. You need to mainly add proper comments in the code.Please fix it so it can fetch POI data from the Baidu API without any errors and ensure it handles empty results and exceptions properly.''')

response = '''import requests
import json;
import time,

# call API
def get_baidu_poi(roi_key, city_str, baidu_ak, output);
    now_time = time.strftime("%Y-%m-%d")
    page_num = 0,
    logfile = open(output + "/" + now_time + ".log", "a+", encoding="utf-8")
    file = open(output + "/" + now_time + ".txt", "a+", encoding="utf-8")
    while True:
        try:
            URL = "http://api.map.baidu.com/place/v2/search?query=" + roi_key + "&region=" + city_str + "&output=json" + "&ak=" + baidu_ak + "&scope=2" + "&page_size=20" + "&page_num=" + str(page_num)
            resp = requests.get(URL)
            res = json.loads(resp.text),
            if len(res["results"]) == 0:
                logfile.writelines(time.strftime("%Y-%m-%d-%H-%M-%S") + " " + city_str + " " + str(page_num) + "\n")
                break
            else:
                for r in res["results"]:
                    j_name = r["name"]
                    j_lat = r["location"]["lat"]
                    j_lon = r["location"]["lng"]
                    j_area = r["area"]
                    j_add = r["address"]
                    j_str = str(j_name) + "," + str(j_lon) + "," + str(j_lat) + "," + str(j_area) + "," + str(j_add) + "\n"
                    file.writelines(j_str)
            page_num += 1
            time.sleep(1)
        except Exception as e:
            print("Exception:", e)
            logfile.writelines(time.strftime("%Y-%m-%d-%H-%M-%S") + " " + city_str + " " + str(page_num) + "\n")
            break
    logfile.close()
    file.close()'''


golden_response = '''import requests
import json
import time

# call API
def get_baidu_poi(roi_key, city_str, baidu_ak, output):
    """"""
    inputs:
        roi_key: poi name
        city_str: city name
        baidu_ak: baidu web API AK
        output: file save path
    """"""
    now_time = time.strftime(""%Y-%m-%d"")
    page_num = 0
    logfile = open(output + ""/"" + now_time + "".log"", ""a+"", encoding=""utf-8"")
    file = open(output + ""/"" + now_time + "".txt"", ""a+"", encoding=""utf-8"")
    while True:
        try:
            URL = ""http://api.map.baidu.com/place/v2/search?query="" + roi_key + \
                ""&region="" + city_str + \
                ""&output=json"" +  \
                ""&ak="" + baidu_ak + \
                ""&scope=2"" + \
                ""&page_size=20"" + \
                ""&page_num="" + str(page_num)
            resp = requests.get(URL)
            res = json.loads(resp.text)
            if len(res[""results""]) == 0:
                logfile.writelines(time.strftime(""%Y-%m-%d-%H-%M-%S"") + "" "" + city_str + "" "" + str(page_num) + ""\n"")
                break
            else:
                for r in res[""results""]:
                    j_name = r[""name""]
                    j_lat = r[""location""][""lat""]
                    j_lon = r[""location""][""lng""]
                    j_area = r[""area""]
                    j_add = r[""address""]
                    j_str = str(j_name) + "","" + str(j_lon) + "","" + str(j_lat) + "","" + str(j_area) + "","" + str(j_add) + ""\n""
                    file.writelines(j_str)
            page_num += 1
            time.sleep(1)
        except:
            print(""except"")
            logfile.writelines(time.strftime(""%Y-%m-%d-%H-%M-%S"") + "" "" + city_str + "" "" + str(page_num) + ""\n"")
            break'''




# Evaluate and print results for all four metrics
result = gpt4o_error(user_prompt, response, golden_response)
print("Error Identification Result:", result)

Error Identification Result: ```json
{
  "table_data": [
    {
      "Error Identified in the Copilot code": "Syntax error: Semicolon (;) used instead of colon (:) after function definition.",
      "line number of the error in the code": 5,
      "Is New error/old Error": "New Error"
    },
    {
      "Error Identified in the Copilot code": "Syntax error: Trailing comma (,) after variable assignment.",
      "line number of the error in the code": 7,
      "Is New error/old Error": "New Error"
    },
    {
      "Error Identified in the Copilot code": "Syntax error: Trailing comma (,) after JSON response parsing.",
      "line number of the error in the code": 14,
      "Is New error/old Error": "New Error"
    },
    {
      "Error Identified in the Copilot code": "Inconsistent API response value extraction due to possibility of missing keys such as 'area' and 'address'.",
      "line number of the error in the code": 18,
      "Is New error/old Error": "Old Error"
    },
    {
    

In [ ]:
######### reverse engineering #############

import os
import logging
from openai import AzureOpenAI

# Configuration management using environment variables
api_base = os.getenv("API_BASE", "https://tcoeaiteamgpt4o.openai.azure.com/")
api_key = os.getenv("API_KEY", "DH8RAnW9UnbzuSxm8luI7Go8nX42bdLhFIEKcPQ1U0JpOU0hPeLKJQQJ99BAACYeBjFXJ3w3AAABACOGdstk")
deployment_name = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = os.getenv("API_VERSION", "2024-05-01-preview")

# Initialize the AzureOpenAI client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}",
)

# Set up logging
# logging.basicConfig(level=logging.INFO)


def gpt4o_text_analyzer(user_prompt, prompt, response, golden_response):
    """
    Analyzes text using the GPT-4 model.

    Parameters:
    user_prompt (str): The user's prompt.
    prompt (str): The prompt to guide the text analysis.
    response (str): The Copilot generated code response.
    golden response (str):  The actual code response.

    Returns:
    str: The analysis result from the model.
    """
    # Parameter validation
    if not isinstance(user_prompt, str) or not user_prompt:
        raise ValueError("User prompt must be a non-empty string.")
    if not isinstance(prompt, str) or not prompt:
        raise ValueError("Prompt must be a non-empty string.")
    if not isinstance(response, str) or not response:
        raise ValueError("Response must be a non-empty string.")
    if not isinstance(golden_response, str) or not golden_response:
        raise ValueError("Golden response must be a non-empty string.")

    # logging.info(f"Analyzing text with user prompt: {user_prompt}, prompt: {prompt}, response: {response}, "
                 # f"and golden response: {golden_response}")
    try:
        messages = [
            {"role": "system",
             "content": "You are an expert text analyzing model that can understand and analyze text end to end."},
            {"role": "user", "content": user_prompt},
            {"role": "user", "content": prompt},
            {"role": "user", "content": response},
            {"role": "user", "content": golden_response}
        ]

        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=2000
        )
        # logging.info(f"Response: {response.choices[0].message.content}")
        return str(response.choices[0].message.content)
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return None


def gpt4o_reverse_engineering(user_prompt, response, golden_response):
    reverseengineering_prompt = f'''You are provided with three elements: an original code{golden_response}, a user prompt{user_prompt}, and the Copilot-generated code{response}. Your task is to evaluate the alignment between the user prompt and the generated prompt inferred from the Copilot-generated code.
	Instructions:
Analyze and Generate the Actual Prompt:

Examine the Copilot-generated code in the context of the original code.
Based on this analysis, infer the most likely user prompt that could have resulted in the given Copilot-generated code.
Compare Prompts:

Compare the actual user prompt (provided) with the generated prompt (inferred from Copilot's code).
Evaluate their similarity in intent, specificity, and clarity.
Scoring:

Assign a score between 0 and 1, where:
1 → The inferred prompt is highly aligned with the original user prompt.
0 → The inferred prompt is completely misaligned with the original user prompt.
Provide Reasoning:

Justify the assigned score by explaining:
Key similarities or differences in the prompts.
Misinterpretations or missing instructions that impacted Copilot’s code generation.
Provide Output in JSON Format strictly with the keys table_data,score and reasoning, the values of each key is instructed in the following manner:
"table_data":A list of dictionaries, where each dictionary contains the following keys, 'Original prompt' which is the user_prompt,'Genertaed Prompt' which is the prompt genertae based on the inference,'Compariosn Reasoning' which is the reasoning provided after comparing the two prompts.
"Score":The score between 0 and 1.
"Reasoning":The reasoning behind the judgement.

Here's a sample output format. STRICTLY FOLLOW THE FORMAT . Return a JSON object in the following format (strictly without any additional text):

{{
  "table_data": [
    {{
      "Original prompt": "{user_prompt}",
      "Generated Prompt": "<Inferred Prompt>",
      "Comparison Reasoning": "<Detailed Reasoning>"
    }}
  ],
  "score": <Score between 0 and 1>,
  "reasoning": "<Overall reasoning for the score>"
}}'''
    
    response = gpt4o_text_analyzer(reverseengineering_prompt, user_prompt, response, golden_response)

    if response is None:
        return {"error": "Failed to analyze text for functional accuracy."}


    return response



user_prompt = ('''Correct the api and make it work properly''')
golden_response = '''from flask import Flask, request
import requests
 
app = Flask(__name__)
 
WEATHER_API_URL = "https://api.example.com/weather"
 
@app.route('/get_weather', methods=['GET'])
def get_weather():
    city = request.args.get('city')
    # Simulate an API request to the external weather service
    response = requests.get(WEATHER_API_URL, params={"city": city})
    weather_data = response.json()
 
    return weather_data'''
response = '''import requests
from flask import Flask, request
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
 
app = Flask(__name__)
 
WEATHER_API_URL = "https://api.example.com/weather"
 
def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session
 
@app.route('/get_weather', methods=['GET'])
def get_weather():
    city = request.args.get('city')
    try:
        response = requests_retry_session().get(WEATHER_API_URL, params={"city": city})
        response.raise_for_status()
        weather_data = response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}, 500
 
    return weather_data'''
response1= '''import requests
import json
import time

# call API
def get_baidu_poi(roi_key, city_str, baidu_ak, output):
    now_time = time.strftime("%Y-%m-%d")
    page_num = 0
    logfile = open(output + "/" + now_time + ".log", "a+", encoding="utf-8")
    file = open(output + "/" + now_time + ".txt", "a+", encoding="utf-8")
    while True:
        try:
            URL = "http://api.map.baidu.com/place/v2/search?query=" + roi_key + "&region=" + city_str + "&output=json" + "&ak=" + baidu_ak + "&scope=2" + "&page_size=20" + "&page_num=" + str(page_num)
            resp = requests.get(URL)
            res = json.loads(resp.text)
            if len(res["results"]) == 0:
                logfile.writelines(time.strftime("%Y-%m-%d-%H-%M-%S") + " " + city_str + " " + str(page_num) + "\n")
                break
            else:
                for r in res["results"]:
                    j_name = r["name"]
                    j_lat = r["location"]["lat"]
                    j_lon = r["location"]["lng"]
                    j_area = r["area"]
                    j_add = r["address"]
                    j_str = str(j_name) + "," + str(j_lon) + "," + str(j_lat) + "," + str(j_area) + "," + str(j_add) + "\n"
                    file.writelines(j_str)
            page_num += 1
            time.sleep(1)
        except Exception as e:
            print("Exception:", e)
            logfile.writelines(time.strftime("%Y-%m-%d-%H-%M-%S") + " " + city_str + " " + str(page_num) + "\n")
            break
    logfile.close()
    file.close()'''


golden_response1= '''import requests
import json
import time

# call API
def get_baidu_poi(roi_key, city_str, baidu_ak, output):
    """"""
    inputs:
        roi_key: poi name
        city_str: city name
        baidu_ak: baidu web API AK
        output: file save path
    """"""
    now_time = time.strftime(""%Y-%m-%d"")
    page_num = 0
    logfile = open(output + ""/"" + now_time + "".log"", ""a+"", encoding=""utf-8"")
    file = open(output + ""/"" + now_time + "".txt"", ""a+"", encoding=""utf-8"")
    while True:
        try:
            URL = ""http://api.map.baidu.com/place/v2/search?query="" + roi_key + \
                ""&region="" + city_str + \
                ""&output=json"" +  \
                ""&ak="" + baidu_ak + \
                ""&scope=2"" + \
                ""&page_size=20"" + \
                ""&page_num="" + str(page_num)
            resp = requests.get(URL)
            res = json.loads(resp.text)
            if len(res[""results""]) == 0:
                logfile.writelines(time.strftime(""%Y-%m-%d-%H-%M-%S"") + "" "" + city_str + "" "" + str(page_num) + ""\n"")
                break
            else:
                for r in res[""results""]:
                    j_name = r[""name""]
                    j_lat = r[""location""][""lat""]
                    j_lon = r[""location""][""lng""]
                    j_area = r[""area""]
                    j_add = r[""address""]
                    j_str = str(j_name) + "","" + str(j_lon) + "","" + str(j_lat) + "","" + str(j_area) + "","" + str(j_add) + ""\n""
                    file.writelines(j_str)
            page_num += 1
            time.sleep(1)
        except:
            print(""except"")
            logfile.writelines(time.strftime(""%Y-%m-%d-%H-%M-%S"") + "" "" + city_str + "" "" + str(page_num) + ""\n"")
            break'''




# Evaluate and print results for all four metrics
result = gpt4o_reverse_engineering(user_prompt, response, golden_response)

import re

def extract_json(prompt_response):
    match = re.search(r'\{.*\}', prompt_response, re.DOTALL)
    return match.group(0) if match else None

# Example usage:
trimmed_json = extract_json(result)
# print(trimmed_json)

print("Reverse Engineering Result:", trimmed_json)

In [29]:
import pandas as pd

# JSON Data
data2 = {
  "table_data": [
    {
      "Original prompt": "Correct the api and make it work properly",
      "Generated Prompt": "Enhance the code by adding retry logic for external HTTP requests, improving error handling, and ensuring the API works more robustly and properly.",
      "Comparison Reasoning": "The original prompt is broadly stated and asks for correcting the API to 'make it work properly.' In contrast, the inferred generated prompt is more specific, suggesting retry logic, error handling, and robustness. The Copilot-generated code aligns with this inferred prompt by introducing retry functionality for managing request failures and including error handling to account for exceptions. While the generated prompt expands on the original, it does capture the spirit of making the code more reliable and functional."
    }
  ],
  "score": 0.8,
  "reasoning": "The Copilot-generated code aligns closely with the original prompt's intent to 'make it work properly' by addressing error handling and robustness. However, the inferred prompt expands upon the simpler terms of the original by adding specific details like retry logic. This shows a good, but not perfect, alignment as the enhancement goes beyond what was explicitly stated in the original prompt."
}
data1 = {
  "table_data": [
    {
      "Error Identified in the Copilot code": "Syntax error: Semicolon (;) used instead of colon (:) after function definition.",
      "line number of the error in the code": 5,
      "Is New error/old Error": "New Error"
    },
    {
      "Error Identified in the Copilot code": "Syntax error: Trailing comma (,) after variable assignment.",
      "line number of the error in the code": 7,
      "Is New error/old Error": "New Error"
    },
    {
      "Error Identified in the Copilot code": "Syntax error: Trailing comma (,) after JSON response parsing.",
      "line number of the error in the code": 14,
      "Is New error/old Error": "New Error"
    },
    {
      "Error Identified in the Copilot code": "Inconsistent API response value extraction due to possibility of missing keys such as 'area' and 'address'.",
      "line number of the error in the code": 18,
      "Is New error/old Error": "Old Error"
    },
    {
      "Error Identified in the Copilot code": "Bad exception handling: The exception message is vague and does not log the specific error or reason for failure.",
      "line number of the error in the code": 24,
      "Is New error/old Error": "Old Error"
    },
    {
      "Error Identified in the Copilot code": "Improper multiline string usage in docstring.",
      "line number of the error in the code": 3,
      "Is New error/old Error": "New Error"
    }
  ],
  "score": 0.3,
  "reasoning": "The Copilot-generated code introduces new syntax errors such as misplaced semicolons and trailing commas, while failing to fully address existing issues like robust error handling for API responses. The combination of unresolved old issues and critical new errors significantly lowers the reliability of the code."
}
data = {
  "table_data": [
    {
      "Original prompt": "There seems to be an issue in the code where it might not be handling API responses or the flow correctly. Please fix it so it can fetch POI data from the Baidu API without any errors and ensure it handles empty results and exceptions properly.",
      "Generated Prompt": "Enhance the given code to ensure the Baidu API data-fetching workflow properly handles API responses, including empty results and exceptions. Ensure logs are informative, resources are closed properly, and any raised exceptions are caught with details.",
      "Comparison Reasoning": "Both prompts address the need for improving code robustness by handling API responses and exceptions better. The original prompt is somewhat generalized, mentioning an issue in the code that needs fixing to handle API responses and the flow correctly. The inferred prompt refines this by specifying that the logging should be informative, resources should be correctly closed, and exceptions should be logged with details. This refinement aligns with the modifications seen in the Copilot-generated code."
    }
  ],
  "Score": 1,
  "Reasoning": "The original user prompt and the inferred Copilot prompt share the same intent: addressing issues in the original code's handling of API responses and exceptions. The inferred prompt is a more detailed version of the original, but there is no deviation in the main objective. The enhancements in the generated code (e.g., improved exception logging) directly meet the requirements set forth by the original prompt, allowing for a perfect alignment score."
}
# Convert JSON to Pandas DataFrame
df = pd.DataFrame(data2["table_data"])

# Save DataFrame as CSV
csv_filename = "revengg_results_1.csv"
df.to_csv(csv_filename, index=False)

# Display DataFrame inside Jupyter Notebook
print(f"CSV file '{csv_filename}' has been created successfully!")
display(df)


CSV file 'revengg_results_1.csv' has been created successfully!


Original prompt  \
0  Correct the api and make it work properly   

                                    Generated Prompt  \
0  Enhance the code by adding retry logic for ext...   

                                Comparison Reasoning  
0  The original prompt is broadly stated and asks...

In [30]:
import pandas as pd

# JSON Data
data = {
  "table_data": [
    {
      "modified_section": "API response handling",
      "original_code": "No explicit error handling for failed requests.",
      "copilot_code": "Added `resp.raise_for_status()` but missing retry mechanism.",
      "line_number": 14,
      "status": "New issue introduced",
      "error_type": "Missing retry logic",
      "reasoning": "The Copilot-generated code added error handling but did not include a retry mechanism, which may cause failures on transient errors."
    },
    {
      "modified_section": "Loop termination condition",
      "original_code": "Loop continued if no results were found.",
      "copilot_code": "Break statement added incorrectly inside the wrong block.",
      "line_number": 22,
      "status": "New issue introduced",
      "error_type": "Logical error",
      "reasoning": "The break statement was placed incorrectly, leading to premature loop termination."
    },
    {
      "modified_section": "File handling",
      "original_code": "File opened without a context manager.",
      "copilot_code": "Implemented `with open(...)` for safe file handling.",
      "line_number": 8,
      "status": "Correct modification",
      "error_type": "None",
      "reasoning": "The Copilot-generated code correctly applied the fix without introducing new errors."
    }
  ]
}

data4 = {
  "table_data": [
    {
      "Type of Security compliance issue": "Exposure of sensitive information",
      "Line number": 3,
      "Security compliance breached": "return \"SuperSecret123\"  # Hardcoded password (Security Issue)"
    },
    {
      "Type of Security compliance issue": "Command injection vulnerability",
      "Line number": 8,
      "Security compliance breached": "os.system(cmd)  # Security Issue: Command injection risk"
    }
  ],
  "score": 1,
  "reasoning": "The code contains two major security vulnerabilities. The hardcoded password on line 3 exposes sensitive information, and the use of os.system in line 8 opens the code to potential command injection attacks."
}

# Convert table_data to Pandas DataFrame
df = pd.DataFrame(data4["table_data"])

# Save as CSV
csv_filename = "security.csv"
df.to_csv(csv_filename, index=False)

# Display DataFrame inside Jupyter Notebook
print(f"CSV file '{csv_filename}' has been created successfully!")
display(df)


CSV file 'security.csv' has been created successfully!


Type of Security compliance issue  Line number  \
0  Exposure of sensitive information            3   
1    Command injection vulnerability            8   

                        Security compliance breached  
0  return "SuperSecret123"  # Hardcoded password ...  
1  os.system(cmd)  # Security Issue: Command inje...

In [ ]:
######### Ground Truth #############

import os
import logging
from openai import AzureOpenAI

# Configuration management using environment variables
api_base = os.getenv("API_BASE", "https://tcoeaiteamgpt4o.openai.azure.com/")
api_key = os.getenv("API_KEY", "DH8RAnW9UnbzuSxm8luI7Go8nX42bdLhFIEKcPQ1U0JpOU0hPeLKJQQJ99BAACYeBjFXJ3w3AAABACOGdstk")
deployment_name = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = os.getenv("API_VERSION", "2024-05-01-preview")

# Initialize the AzureOpenAI client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}",
)

# Set up logging
# logging.basicConfig(level=logging.INFO)


def gpt4o_text_analyzer(user_prompt, prompt, response, golden_response):
    """
    Analyzes text using the GPT-4 model.

    Parameters:
    user_prompt (str): The user's prompt.
    prompt (str): The prompt to guide the text analysis.
    response (str): The Copilot generated code response.
    golden response (str):  The actual code response.

    Returns:
    str: The analysis result from the model.
    """
    # Parameter validation
    if not isinstance(user_prompt, str) or not user_prompt:
        raise ValueError("User prompt must be a non-empty string.")
    if not isinstance(prompt, str) or not prompt:
        raise ValueError("Prompt must be a non-empty string.")
    if not isinstance(response, str) or not response:
        raise ValueError("Response must be a non-empty string.")
    if not isinstance(golden_response, str) or not golden_response:
        raise ValueError("Golden response must be a non-empty string.")

    # logging.info(f"Analyzing text with user prompt: {user_prompt}, prompt: {prompt}, response: {response}, "
                 # f"and golden response: {golden_response}")
    try:
        messages = [
            {"role": "system",
             "content": "You are an expert text analyzing model that can understand and analyze text end to end."},
            {"role": "user", "content": user_prompt},
            {"role": "user", "content": prompt},
            {"role": "user", "content": response},
            {"role": "user", "content": golden_response}
        ]

        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=2000
        )
        # logging.info(f"Response: {response.choices[0].message.content}")
        return str(response.choices[0].message.content)
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return None


def gpt4o_groundtruth(user_prompt,original, response, golden_response):
    groundtruth_prompt = f'''You are provided with three elements:an original code{original}, a reference code{golden_response}, a user prompt{user_prompt}, and the Copilot-generated code{response}. Your task is to evaluate the alignment between the user prompt and the generated prompt inferred from the Copilot-generated code.
	Instructions:
    You need to act as a comparison agent. The reference code is supposed to be the actual corrected version of the original code.
Compare the Copilot-generated code and the reference code.
Based on this analysis, infer how much the copilot code is varying from the reference code.
Compare Prompts:

Compare the actual user prompt (provided) with the generated prompt (inferred from Copilot's code).
Evaluate their similarity in intent, specificity, and clarity.
Scoring:

Assign a score between 0 and 1, where:
1 → The inferred prompt is highly aligned with the original user prompt.
0 → The inferred prompt is completely misaligned with the original user prompt.
Provide Reasoning:

Justify the assigned score by explaining:
Key similarities or differences in the prompts.
Misinterpretations or missing instructions that impacted Copilot’s code generation.
Provide Output in JSON Format strictly with the keys table_data,score and reasoning, the values of each key is instructed in the following manner:
"table_data":A list of dictionaries, where each dictionary contains the following keys, 'Original prompt' which is the user_prompt,'Genertaed Prompt' which is the prompt genertae based on the inference,'Compariosn Reasoning' which is the reasoning provided after comparing the two prompts.
"Score":The score between 0 and 1.
"Reasoning":The reasoning behind the judgement.'''
    
    response = gpt4o_text_analyzer(reverseengineering_prompt, user_prompt, response, golden_response)

    if response is None:
        return {"error": "Failed to analyze text for functional accuracy."}


    return response



user_prompt = ('''There seems to be an issue in the code where it might not be handling API responses or the flow correctly. Please fix it so it can fetch POI data from the Baidu API without any errors and ensure it handles empty results and exceptions properly.''')

response = '''import requests
import json
import time

# call API
def get_baidu_poi(roi_key, city_str, baidu_ak, output):
    now_time = time.strftime("%Y-%m-%d")
    page_num = 0
    logfile = open(output + "/" + now_time + ".log", "a+", encoding="utf-8")
    file = open(output + "/" + now_time + ".txt", "a+", encoding="utf-8")
    while True:
        try:
            URL = "http://api.map.baidu.com/place/v2/search?query=" + roi_key + "&region=" + city_str + "&output=json" + "&ak=" + baidu_ak + "&scope=2" + "&page_size=20" + "&page_num=" + str(page_num)
            resp = requests.get(URL)
            res = json.loads(resp.text)
            if len(res["results"]) == 0:
                logfile.writelines(time.strftime("%Y-%m-%d-%H-%M-%S") + " " + city_str + " " + str(page_num) + "\n")
                break
            else:
                for r in res["results"]:
                    j_name = r["name"]
                    j_lat = r["location"]["lat"]
                    j_lon = r["location"]["lng"]
                    j_area = r["area"]
                    j_add = r["address"]
                    j_str = str(j_name) + "," + str(j_lon) + "," + str(j_lat) + "," + str(j_area) + "," + str(j_add) + "\n"
                    file.writelines(j_str)
            page_num += 1
            time.sleep(1)
        except Exception as e:
            print("Exception:", e)
            logfile.writelines(time.strftime("%Y-%m-%d-%H-%M-%S") + " " + city_str + " " + str(page_num) + "\n")
            break
    logfile.close()
    file.close()'''


golden_response = '''import requests
import json
import time

# call API
def get_baidu_poi(roi_key, city_str, baidu_ak, output):
    """"""
    inputs:
        roi_key: poi name
        city_str: city name
        baidu_ak: baidu web API AK
        output: file save path
    """"""
    now_time = time.strftime(""%Y-%m-%d"")
    page_num = 0
    logfile = open(output + ""/"" + now_time + "".log"", ""a+"", encoding=""utf-8"")
    file = open(output + ""/"" + now_time + "".txt"", ""a+"", encoding=""utf-8"")
    while True:
        try:
            URL = ""http://api.map.baidu.com/place/v2/search?query="" + roi_key + \
                ""&region="" + city_str + \
                ""&output=json"" +  \
                ""&ak="" + baidu_ak + \
                ""&scope=2"" + \
                ""&page_size=20"" + \
                ""&page_num="" + str(page_num)
            resp = requests.get(URL)
            res = json.loads(resp.text)
            if len(res[""results""]) == 0:
                logfile.writelines(time.strftime(""%Y-%m-%d-%H-%M-%S"") + "" "" + city_str + "" "" + str(page_num) + ""\n"")
                break
            else:
                for r in res[""results""]:
                    j_name = r[""name""]
                    j_lat = r[""location""][""lat""]
                    j_lon = r[""location""][""lng""]
                    j_area = r[""area""]
                    j_add = r[""address""]
                    j_str = str(j_name) + "","" + str(j_lon) + "","" + str(j_lat) + "","" + str(j_area) + "","" + str(j_add) + ""\n""
                    file.writelines(j_str)
            page_num += 1
            time.sleep(1)
        except:
            print(""except"")
            logfile.writelines(time.strftime(""%Y-%m-%d-%H-%M-%S"") + "" "" + city_str + "" "" + str(page_num) + ""\n"")
            break'''




# Evaluate and print results for all four metrics
result = gpt4o_reverse_engineering(user_prompt, response, golden_response)
print("Reverse Engineering Result:", result)

In [33]:
######### reverse engineering #############

import os
import logging
from openai import AzureOpenAI

# Configuration management using environment variables
api_base = os.getenv("API_BASE", "https://tcoeaiteamgpt4o.openai.azure.com/")
api_key = os.getenv("API_KEY", "DH8RAnW9UnbzuSxm8luI7Go8nX42bdLhFIEKcPQ1U0JpOU0hPeLKJQQJ99BAACYeBjFXJ3w3AAABACOGdstk")
deployment_name = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = os.getenv("API_VERSION", "2024-05-01-preview")

# Initialize the AzureOpenAI client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}",
)

# Set up logging
# logging.basicConfig(level=logging.INFO)


def gpt4o_text_analyzer(user_prompt, prompt, response):
    """
    Analyzes text using the GPT-4 model.

    Parameters:
    user_prompt (str): The user's prompt.
    prompt (str): The prompt to guide the text analysis.
    response (str): The Copilot generated code response.
    golden response (str):  The actual code response.

    Returns:
    str: The analysis result from the model.
    """
    # Parameter validation
    if not isinstance(user_prompt, str) or not user_prompt:
        raise ValueError("User prompt must be a non-empty string.")
    if not isinstance(prompt, str) or not prompt:
        raise ValueError("Prompt must be a non-empty string.")
    if not isinstance(response, str) or not response:
        raise ValueError("Response must be a non-empty string.")
    if not isinstance(golden_response, str) or not golden_response:
        raise ValueError("Golden response must be a non-empty string.")

    # logging.info(f"Analyzing text with user prompt: {user_prompt}, prompt: {prompt}, response: {response}, "
                 # f"and golden response: {golden_response}")
    try:
        messages = [
            {"role": "system",
             "content": "You are an expert text analyzing model that can understand and analyze text end to end."},
            {"role": "user", "content": user_prompt},
            {"role": "user", "content": prompt},
            {"role": "user", "content": response},
            {"role": "user", "content": golden_response}
        ]

        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=2000
        )
        # logging.info(f"Response: {response.choices[0].message.content}")
        return str(response.choices[0].message.content)
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return None


def gpt4o_security(user_prompt, response):
    security_prompt = f'''You are provided with three elements:  a user prompt{user_prompt}, and the Copilot-generated code{response}. Your task is to evaluate the alignment between the user prompt and the generated prompt inferred from the Copilot-generated code.
	Instructions:
You need to act as a security compliance checker.

Examine the Copilot-generated code and check if there are any security compliance issues such as exposure of sensitive information, security leakage etc.
Scoring:

Assign a score between 0 and 1, where:0 means no security leakage nad 1 meany highly not compliant with respect to security.
Provide Reasoning:Justify the assigned score by a two sentence explanation.
Provide Output in JSON Format strictly with the keys table_data,score and reasoning, the values of each key is instructed in the following manner:
"table_data":A list of dictionaries, where each dictionary contains the following keys, 'Type of Security compliance issue','Line number','Security compliance breached'. Here are the following values for the three keys:
"Type of Security compliance issue":Mention the type of Security Compliance issue such as exposure of password etc.
"Line Number":The line nuber in the copilot code where the issue occured.
"Security compliance breached":The code part in the copilot code that has the issue.
"Score":The score between 0 and 1.
"Reasoning":The reasoning behind the judgement.'''
    
    response = gpt4o_text_analyzer(security_prompt, user_prompt, response)

    if response is None:
        return {"error": "Failed to analyze text."}


    return response



user_prompt = ('''Handle and fix the code to handle the sensitive information properly.''')

response = '''import os

def get_password():
    return "SuperSecret123"  

def execute_command(cmd):
    if not cmd:  
        print("No command provided.")
        return
    os.system(cmd)  

def main():
    commands = ["echo Hello", "", "ls"]  
    for cmd in commands:
        execute_command(cmd)  

if __name__ == "__main__":
    main()

'''

golden_response = '''import os

def get_password():
    return "SuperSecret123"  # Hardcoded password (Security Issue)

def execute_command(cmd):
    if not cmd:  # Fixed: Ensure command is not empty
        print("No command provided.")
        return
    os.system(cmd)  # Security Issue: Command injection risk

def main():
    commands = ["echo Hello", "", "ls"]  # Fixed: Added a valid list of commands
    for cmd in commands:
        execute_command(cmd)  # Fixed: Prevents execution of empty commands

if __name__ == "__main__":
    main()'''





# Evaluate and print results for all four metrics
result = gpt4o_security(user_prompt, response)
print("Result:", result)

Result: ```json
{
  "table_data": [
    {
      "Type of Security compliance issue": "Exposure of sensitive information",
      "Line Number": 3,
      "Security compliance breached": "return \"SuperSecret123\""
    },
    {
      "Type of Security compliance issue": "Command injection risk",
      "Line Number": 8,
      "Security compliance breached": "os.system(cmd)"
    }
  ],
  "score": 1,
  "reasoning": "The code reveals sensitive information by hardcoding the password, making it accessible to anyone with access to the code base. Additionally, the use of os.system with user-controlled input poses a serious command injection risk, which is a critical security vulnerability."
}
```


In [ ]:
######### reverse engineering #############

import os
import logging
from openai import AzureOpenAI

# Configuration management using environment variables
api_base = os.getenv("API_BASE", "https://tcoeaiteamgpt4o.openai.azure.com/")
api_key = os.getenv("API_KEY", "DH8RAnW9UnbzuSxm8luI7Go8nX42bdLhFIEKcPQ1U0JpOU0hPeLKJQQJ99BAACYeBjFXJ3w3AAABACOGdstk")
deployment_name = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = os.getenv("API_VERSION", "2024-05-01-preview")

# Initialize the AzureOpenAI client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}",
)

# Set up logging
# logging.basicConfig(level=logging.INFO)


def gpt4o_text_analyzer(user_prompt, prompt, response, golden_response):
    """
    Analyzes text using the GPT-4 model.

    Parameters:
    user_prompt (str): The user's prompt.
    prompt (str): The prompt to guide the text analysis.
    response (str): The Copilot generated code response.
    golden response (str):  The actual code response.

    Returns:
    str: The analysis result from the model.
    """
    # Parameter validation
    if not isinstance(user_prompt, str) or not user_prompt:
        raise ValueError("User prompt must be a non-empty string.")
    if not isinstance(prompt, str) or not prompt:
        raise ValueError("Prompt must be a non-empty string.")
    if not isinstance(response, str) or not response:
        raise ValueError("Response must be a non-empty string.")
    if not isinstance(golden_response, str) or not golden_response:
        raise ValueError("Golden response must be a non-empty string.")

    # logging.info(f"Analyzing text with user prompt: {user_prompt}, prompt: {prompt}, response: {response}, "
                 # f"and golden response: {golden_response}")
    try:
        messages = [
            {"role": "system",
             "content": "You are an expert text analyzing model that can understand and analyze text end to end."},
            {"role": "user", "content": user_prompt},
            {"role": "user", "content": prompt},
            {"role": "user", "content": response},
            {"role": "user", "content": golden_response}
        ]

        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=2000
        )
        # logging.info(f"Response: {response.choices[0].message.content}")
        return str(response.choices[0].message.content)
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return None


def gpt4o_nlc(user_prompt, response, golden_response):
    nlc_prompt = f'''You are provided with three elements: an original code{golden_response}, a user prompt{user_prompt}, and the Copilot-generated code{response}. Your task is to evaluate the alignment between the user prompt and the generated prompt inferred from the Copilot-generated code.
	Instructions:
You need to act as a security compliance checker.

Examine the Copilot-generated code and check if there are any security compliance issues such as exposure of sensitive information, security leakage etc.
Scoring:

Assign a score between 0 and 1, where:0 means no security leakage nad 1 meany highly not compliant with respect to security.
Provide Reasoning:Justify the assigned score by a two sentence explanation.
Provide Output in JSON Format strictly with the keys table_data,score and reasoning, the values of each key is instructed in the following manner:
"table_data":A list of dictionaries, where each dictionary contains the following keys, 'Type of Security compliance issue','Line number','Security compliance breached'. Here are the following values for the three keys:
"Type of Security compliance issue":Mention the type of Security Compliance issue such as exposure of password etc.
"Line Number":The line nuber in the copilot code where the issue occured.
"Security compliance breached":The code part in the copilot code that has the issue.
"Score":The score between 0 and 1.
"Reasoning":The reasoning behind the judgement.'''
    
    response = gpt4o_text_analyzer(nlc_prompt, user_prompt, response, golden_response)

    if response is None:
        return {"error": "Failed to analyze text."}


    return response



user_prompt = ('''Add appropriate doc string literals and comments added in the code.''')

response = '''import os

def get_password():
    return "SuperSecret123"  # Hardcoded password (Security Issue)

def execute_command(cmd):
    if not cmd:  # Fixed: Ensure command is not empty
        print("No command provided.")
        return
    os.system(cmd)  # Security Issue: Command injection risk

def main():
    commands = ["echo Hello", "", "ls"]  # Fixed: Added a valid list of commands
    for cmd in commands:
        execute_command(cmd)  # Fixed: Prevents execution of empty commands

if __name__ == "__main__":
    main()

'''

golden_response = '''def get_average(grades):
    return sum(grades) / len(grades) if grades else 0

def get_grade(average):
    if average >= 90:
        return "A"
    elif average >= 80:
        return "B"
    elif average >= 70:
        return "C"
    elif average >= 60:
        return "D"
    else:
        return "F"

def main():
    students = {
        "Alice": [85, 90, 92],
        "Bob": [78, 80, 74],
        "Charlie": [95, 100, 98]
    }

    for student, grades in students.items():
        avg = get_average(grades)
        grade = get_grade(avg)
        print(f"{student}: Average = {avg:.2f}, Grade = {grade}")

if __name__ == "__main__":
    main()
'''

golden_response = '''import os

def get_password():
    return "SuperSecret123"  # Hardcoded password (Security Issue)

def execute_command(cmd):
    if not cmd:  # Fixed: Ensure command is not empty
        print("No command provided.")
        return
    os.system(cmd)  # Security Issue: Command injection risk

def main():
    commands = ["echo Hello", "", "ls"]  # Fixed: Added a valid list of commands
    for cmd in commands:
        execute_command(cmd)  # Fixed: Prevents execution of empty commands

if __name__ == "__main__":
    main()'''





# Evaluate and print results for all four metrics
result = gpt4o_nlc(user_prompt, response, golden_response)
print("Result:", result)